### Newsbot to write a daily AI news summary using langgraph
- Save a list of HTML files from sources.yaml (tech news sites)
- Extract URLs for the news stories
- Filter URLs to remove duplicates, articles seen before, and non-AI articles (using a ChatGPT prompt)
- Perform headline topic analysis and sort by topic to help the AI structure the response by topic
- Scrape and summarize individual articles
- Compose and email the summary


In [1]:
# to selectively re-import as needed
# import sys
# del sys.modules['ainb_llm']
# del sys.modules['ainb_const']
# del sys.modules['ainb_utilities']
# del sys.modules['ainb_webscrape']


In [2]:
from datetime import datetime
import os
import yaml
import sqlite3
import json
from collections import Counter
import uuid
from typing import TypedDict, Annotated

import operator
import pickle

import langchain
from langchain_openai import ChatOpenAI
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_core.prompts import (ChatPromptTemplate, MessagesPlaceholder, PromptTemplate,
                                    SystemMessagePromptTemplate, HumanMessagePromptTemplate)
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver

from langchain_core.output_parsers import SimpleJsonOutputParser, JsonOutputParser, StrOutputParser

from langgraph.graph import StateGraph, START, END

import numpy as np
import pandas as pd
import umap

import sklearn
from sklearn.cluster import DBSCAN

import bs4
from urllib.parse import urljoin, urlparse

import multiprocessing
import asyncio
import aiohttp

from IPython.display import HTML, Image, Markdown, display
import markdown

import openai
from openai import OpenAI

import trafilatura

VERBOSE=1
from ainb_const import (DOWNLOAD_DIR, PAGES_DIR,
                        MODEL, LOWCOST_MODEL, HIGHCOST_MODEL, CANONICAL_TOPICS,
                        SOURCECONFIG, FILTER_PROMPT, TOPIC_PROMPT,
                        SUMMARIZE_SYSTEM_PROMPT, SUMMARIZE_USER_PROMPT, FINAL_SUMMARY_PROMPT, TOP_CATEGORIES_PROMPT,
                        MAX_INPUT_TOKENS, MAX_OUTPUT_TOKENS, MAX_RETRIES, TEMPERATURE, SQLITE_DB,
                        HOSTNAME_SKIPLIST, SITE_NAME_SKIPLIST,
                       )
from ainb_utilities import (log, delete_files, filter_unseen_urls_db, insert_article,
                            nearest_neighbor_sort, agglomerative_cluster_sort, traveling_salesman_sort_scipy,
                            unicode_to_ascii, send_gmail)
from ainb_webscrape import (get_driver, quit_drivers, launch_drivers, get_file, get_url, parse_file,
                            get_og_tags, get_path_from_url, trimmed_href, process_source_queue_factory,
                            process_url_queue_factory, get_google_news_redirects)
from ainb_llm import (paginate_df, process_pages, fetch_pages, fetch_openai, fetch_all_summaries,
                      fetch_openai_summary, count_tokens, trunc_tokens, 
                      categorize_headline, categorize_df, clean_topics,
                      get_site_name, fetch_missing_site_names)


import asyncio
# need this to run async in jupyter since it already has an asyncio event loop running
import nest_asyncio
nest_asyncio.apply()


In [3]:
print(f"LangChain         {langchain.__version__}")
print(f"OpenAI            {openai.__version__}")
# print(f"smtplib           {smtplib.sys.version}")
print(f"trafilatura       {trafilatura.__version__}")
# print(f"bs4               {bs4.__version__}")
print(f"numpy             {np.__version__}")
print(f"pandas            {pd.__version__}")
print(f"sklearn           {sklearn.__version__}")
print(f"umap              {umap.__version__}")


LangChain         0.3.0
OpenAI            1.47.0
trafilatura       1.12.2
numpy             1.26.4
pandas            2.2.3
sklearn           1.5.2
umap              0.5.6


In [4]:
# a basic LLM call with langchain
model = ChatOpenAI(model=MODEL)

model.invoke([
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content='Listen to me. You are beautiful. You are perfect and I love you.'),
])


2024-10-02 18:01:06,127 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='Ascoltami. Sei bellissima. Sei perfetta e ti amo.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 34, 'total_tokens': 50, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'chatgpt-4o-latest', 'system_fingerprint': 'fp_bd428e1c3b', 'finish_reason': 'stop', 'logprobs': None}, id='run-0e8c64e5-f919-47f4-8e35-ccd9138dc7e6-0', usage_metadata={'input_tokens': 34, 'output_tokens': 16, 'total_tokens': 50})

In [5]:
# use template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
parser = StrOutputParser()
chain = prompt_template | model | parser
chain.invoke({"language": "italian", "text": "hi"})


2024-10-02 18:01:06,444 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'Ciao'

In [6]:
prompt_inputs = [
    {"language": "French", "adjective1": "flawless", "adjective2": "beautiful"},
    {"language": "German", "adjective1": "immaculate", "adjective2": "exquisite"},
    {"language": "Spanish", "adjective1": "perfect", "adjective2": "gorgeous"},
    {"language": "Italian", "adjective1": "amazing", "adjective2": "magnificent"},
    {"language": "Hungarian", "adjective1": "ravishing", "adjective2": "stunning"},
]

system_template = 'Translate the following into {language}:'
user_template = 'Listen to me. You are {adjective1}. You are {adjective2} and I love you.'

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template),
     ("user", user_template)]
)

parser = StrOutputParser()

chain = prompt_template | model | parser

start_time = datetime.now()
for tpl in prompt_inputs:
    response = ""
    # stream tokens as they are generated
    for r in chain.stream(tpl):
        print(r, end="")
        response += r
end_time = datetime.now()

difference = end_time - start_time
total_seconds = difference.total_seconds()
print(f"\n\nElapsed seconds: {total_seconds:.6f}")


2024-10-02 18:01:06,799 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Écoute-moi. Tu es parfait(e). Tu es magnifique et je t'aime.

2024-10-02 18:01:07,455 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Hör mir zu. Du bist makellos. Du bist exquisit und ich liebe dich.

2024-10-02 18:01:08,648 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Escúchame. Eres perfecto/a. Eres precioso/a y te amo.

2024-10-02 18:01:10,314 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Ascoltami. Sei incredibile. Sei magnifica e ti amo.

2024-10-02 18:01:11,159 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Figyelj rám. Elbűvölő vagy. Lenyűgöző vagy, és szeretlek.

Elapsed seconds: 5.059688


In [7]:
# same but send all at once using asyncio

async def async_langchain(chain, input_dict):
    response = await chain.ainvoke(input_dict)
    return response


prompt_templates = [
    {"language": "French", "adjective1": "flawless", "adjective2": "beautiful"},
    {"language": "German", "adjective1": "immaculate", "adjective2": "exquisite"},
    {"language": "Spanish", "adjective1": "perfect", "adjective2": "gorgeous"},
    {"language": "Italian", "adjective1": "amazing", "adjective2": "magnificent"},
    {"language": "Hungarian", "adjective1": "ravishing", "adjective2": "stunning"},
]

start_time = datetime.now()
tasks = []
for d in prompt_templates:
    task = asyncio.create_task(async_langchain(chain, d))
    tasks.append(task)
responses = await asyncio.gather(*tasks)
end_time = datetime.now()


difference = end_time - start_time
total_seconds = difference.total_seconds()
print(f"\n\nElapsed seconds: {total_seconds:.6f}")
print("\n".join(responses))


2024-10-02 18:01:12,162 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-02 18:01:12,578 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-02 18:01:12,582 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-02 18:01:13,136 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-02 18:01:13,241 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"




Elapsed seconds: 1.718811
Écoute-moi. Tu es parfait(e). Tu es magnifique et je t'aime.
Hör mir zu. Du bist makellos. Du bist exquisit und ich liebe dich.
Escúchame. Eres perfecta. Eres hermosa y te amo.
Ascoltami. Sei incredibile. Sei magnifico/a e ti amo.
Figyelj rám. Elbűvölő vagy. Lenyűgöző vagy, és szeretlek.


In [8]:
DEBUG = True
N_BROWSERS = 4
before_date = None
do_download = True
# before_date="2024-09-29 10:00:00"


In [9]:
# class to maintain settings and state within graph

class AgentState(TypedDict):
    AIdf: dict                #  the current working set of headlines (pandas dataframe not supported)
    before_date: str          #  ignore stories before this date for deduplication (force reprocess since)
    do_download: bool         #  if False use existing files, else download from sources
    sources: dict             #  sources to scrap
    sources_reverse: dict     #  map file names to sources
    headline_str: str         #  headline text email
    bullets: str              #  bullet point summary email
    summary: str              #  final summary
    n_edits: int              #  count edit iterations so we don't keep editing forever
    edit_complete: bool       #  edit will update if no more edits to make
    # message thread with OpenAI
    messages: Annotated[list[AnyMessage], operator.add]


In [10]:
# Initialize reading configurations from YAML file

def fn_initialize(state: AgentState) -> AgentState:
    """
    Initializes the agent state by loading source configurations from SOURCECONFIG (sources.yaml) .

    Args:
        state (AgentState): The current state of the agent.
        verbose (bool, optional): Whether to print verbose output. Defaults to False.

    Returns:
        AgentState: The updated state of the agent.

    Raises:
        yaml.YAMLError: If there is an error while loading the YAML file.

    """

    #  load sources to scrape from sources.yaml
    with open(SOURCECONFIG, "r") as stream:
        try:
            state['sources'] = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

    log(f"Initialized {len(state['sources'])} items in sources from {SOURCECONFIG}")

    # make a reverse dict to map file titles to source names
    state['sources_reverse'] = {}
    for k, v in state['sources'].items():
        log(f"{k} -> {v['url']} -> {v['title']}.html")
        v['sourcename'] = k
        # map filename (title) to source name
        state['sources_reverse'][v['title']] = k

    log(f"Initialized {len(state['sources_reverse'])} items in sources_reverse")

    return state


if DEBUG:
    test_state = AgentState()
    fn_initialize(test_state)


2024-10-02 18:01:13,272 - AInewsbot - INFO - Initialized 17 items in sources from sources.yaml
2024-10-02 18:01:13,273 - AInewsbot - INFO - Ars Technica -> https://arstechnica.com/ -> Ars Technica.html
2024-10-02 18:01:13,273 - AInewsbot - INFO - Bloomberg Tech -> https://www.bloomberg.com/ai -> Bloomberg Technology.html
2024-10-02 18:01:13,273 - AInewsbot - INFO - Business Insider -> https://www.businessinsider.com/tech -> Business Insider Tech.html
2024-10-02 18:01:13,274 - AInewsbot - INFO - FT Tech -> https://www.ft.com/technology -> FT Technology.html
2024-10-02 18:01:13,274 - AInewsbot - INFO - Feedly AI -> https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMiLCJzYWxpZW5jZSI6ImFib3V0In1dLCJidW5kbGVzIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC9jYXRlZ29yeS9HYWRnZXRzIn1dfQ -> Feedly AI.html
2024-10-02 18:01:13,274 - AInewsbot - INFO - 

In [11]:
# scrape sources with selenium and save local files in DOWNLOAD_DIR (htmldata)
def fn_download_sources(state: AgentState) -> AgentState:
    """
    Scrapes sources and saves HTML files.
    If state["do_download"] is True, deletes all files in DOWNLOAD_DIR (htmldata) and scrapes fresh copies.
    If state["do_download"] is False, uses existing files in DOWNLOAD_DIR.
    Uses state["sources"] for config info on sources to scrape
    For each source, saves the current filename to state["sources"][sourcename]['latest']

    Args:
        state (AgentState): The current state of the agent.
        do_delete (bool, optional): Whether to delete files in DOWNLOAD_DIR. Defaults to True.

    Returns:
        AgentState: The updated state of the agent.
    """

    if state["do_download"]:
        # empty download directory
        delete_files(DOWNLOAD_DIR)

        # save each file specified from sources
        log(f"Saving HTML files using {N_BROWSERS} browsers")

        # Create a queue for multiprocessing and populate it
        queue = multiprocessing.Queue()
        for item in state["sources"].values():
            queue.put(item)

        # Function to take the queue and pop entries off and process until none are left
        # lets you create an array of functions with different args
        callable = process_source_queue_factory(queue)

        saved_pages = launch_drivers(N_BROWSERS, callable)
        for sourcename, file in saved_pages:
            log(f"Downloaded {sourcename} to {file}")
            state['sources'][sourcename]['latest'] = file
        log(f"Saved {len(saved_pages)} HTML files")

    else:   # use existing files
        log(f"Web fetch disabled, using existing files in {DOWNLOAD_DIR}")
        # Get the current date
        datestr = datetime.now().strftime("%m_%d_%Y")
        files = [os.path.join(DOWNLOAD_DIR, file)
                 for file in os.listdir(DOWNLOAD_DIR)]
        # filter files with today's date ending in .html
        files = [
            file for file in files if datestr in file and file.endswith(".html")]
        log(f"Found {len(files)} previously downloaded files")
        for file in files:
            log(file)

        saved_pages = []
        for file in files:
            filename = os.path.basename(file)
            # locate date like '01_14_2024' in filename
            position = filename.find(" (" + datestr)
            basename = filename[:position]
            # match to source name
            sourcename = state["sources_reverse"].get(basename)
            if sourcename is None:
                log(f"Skipping {basename}, no sourcename metadata")
                continue
            state["sources"][sourcename]['latest'] = file

    return state

if DEBUG:
    test_state["do_download"] = do_download
    _ = fn_download_sources(test_state)


2024-10-02 18:01:13,287 - AInewsbot - INFO - Saving HTML files using 4 browsers
2024-10-02 18:01:13,299 - AInewsbot - INFO - get_driver - 25581 Initializing webdriver
2024-10-02 18:01:13,300 - AInewsbot - INFO - get_driver - 25581 Initializing webdriver
2024-10-02 18:01:13,301 - AInewsbot - INFO - get_driver - 25581 Initializing webdriver
2024-10-02 18:01:13,301 - AInewsbot - INFO - get_driver - 25581 Initializing webdriver
2024-10-02 18:01:29,644 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-10-02 18:01:29,644 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-10-02 18:01:29,644 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-10-02 18:01:29,644 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-10-02 18:01:29,645 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-10-02 18:01:29,645 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-10-02 18:01:29,645 - AInewsbot - INFO -

2024-10-02 18:02:51,336 - AInewsbot - INFO - get_files(Google News - Technology - Artificial intelligence) - Loading additional infinite scroll items
2024-10-02 18:03:01,367 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:510:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

2024-10-02 18:03:01,368 - AInewsbot - INFO - get_files(HackerNoon) - Saving HackerNoon (10_02_2024 06_03_01 PM).html as utf-8
2024-10-02 18:03:01,374 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit:

2024-10-02 18:03:36,331 - AInewsbot - INFO - get_files(Washington Post Technology) - Saving Washington Post Technology (10_02_2024 06_03_36 PM).html as utf-8
2024-10-02 18:03:36,333 - AInewsbot - INFO - Quit webdriver
2024-10-02 18:03:40,332 - AInewsbot - INFO - get_files(Feedly AI) - Loading additional infinite scroll items
2024-10-02 18:03:50,344 - AInewsbot - INFO - get_files(Feedly AI) - Loading additional infinite scroll items
2024-10-02 18:04:00,376 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:510:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

20

In [12]:
def fn_extract_urls(state: AgentState) -> AgentState:
    """
    Extracts news URLs from the latest HTML files matching the patterns defined in the state['sources'] configuration info.

    Args:
        state (AgentState): The current state of the agent.

    Returns:
        AgentState: The updated state of the agent with the extracted URLs stored in state['AIdf'].
    """
    # Parse news URLs and titles from downloaded HTML files
    log("Parsing html files")
    all_urls = []
    for sourcename, sourcedict in state['sources'].items():
        filename = sourcedict.get('latest')
        if not filename:
            log(f"no filename found for {sourcename}")
            continue

        log(sourcename + ' -> ' + filename)
        links = parse_file(state['sources'][sourcename])
        log(f"{len(links)} links found")
        all_urls.extend(links)

    log(f"Saved {len(all_urls)} links")

    # make a pandas dataframe of all the links found
    AIdf = (
        pd.DataFrame(all_urls)
        .groupby("url")
        .first()
        .reset_index()
        .sort_values("src")[["src", "title", "url"]]
        .reset_index(drop=True)
        .reset_index(drop=False)
        .rename(columns={"index": "id"})
    )
    state['AIdf'] = AIdf.to_dict()

    return state


if DEBUG:
    _ = fn_extract_urls(test_state)

    # s/b 17 but if some like bloomberg" or wsj are missing, maybe got a robot block
    # in that case download missing ones manually and then re-run extract
    print(len(pd.DataFrame(test_state["AIdf"]).groupby('src').count()))
    display(pd.DataFrame(test_state["AIdf"]).groupby('src').count()[['id']])


2024-10-02 18:04:02,177 - AInewsbot - INFO - Parsing html files
2024-10-02 18:04:02,178 - AInewsbot - INFO - Ars Technica -> htmldata/Ars Technica (10_02_2024 06_02_39 PM).html
2024-10-02 18:04:02,251 - AInewsbot - INFO - parse_file - found 172 raw links
2024-10-02 18:04:02,256 - AInewsbot - INFO - parse_file - found 27 filtered links
2024-10-02 18:04:02,256 - AInewsbot - INFO - 27 links found
2024-10-02 18:04:02,256 - AInewsbot - INFO - Bloomberg Tech -> htmldata/Bloomberg Technology (10_02_2024 06_02_40 PM).html
2024-10-02 18:04:02,275 - AInewsbot - INFO - parse_file - found 198 raw links
2024-10-02 18:04:02,278 - AInewsbot - INFO - parse_file - found 16 filtered links
2024-10-02 18:04:02,278 - AInewsbot - INFO - 16 links found
2024-10-02 18:04:02,278 - AInewsbot - INFO - Business Insider -> htmldata/Business Insider Tech (10_02_2024 06_02_39 PM).html
2024-10-02 18:04:02,301 - AInewsbot - INFO - parse_file - found 309 raw links
2024-10-02 18:04:02,305 - AInewsbot - INFO - parse_file 

17


,id
src,
Ars Technica,27
Bloomberg Tech,13
Business Insider,50
FT Tech,60
Feedly AI,55
Google News,66
Hacker News,28
Hacker News 2,19
HackerNoon,104


In [13]:
# sqlite schema to store previously processed 
# CREATE TABLE news_articles (
#     id INTEGER PRIMARY KEY,
#     src TEXT,
#     title TEXT,
#     url TEXT UNIQUE,
#     isAI BOOLEAN,
#     article_date DATE
# , timestamp DATETIME, actual_url TEXT, actual_src TEXT);
#
# CREATE TABLE sites (
#     id INTEGER PRIMARY KEY AUTOINCREMENT,
#     hostname TEXT UNIQUE NOT NULL,
#     site_name TEXT NOT NULL
# );



In [14]:
# filter and clean URLs for new AI stories

def fn_filter_urls(state: AgentState) -> AgentState:
    """
    Filters the URLs in state["AIdf"] to include only those that have not been previously seen,
    and are related to AI according to the response from a ChatGPT prompt.

    Args:
        state (AgentState): The current state of the agent.
        before_date (str, optional): The date before which the URLs should be filtered. Defaults to "".

    Returns:


        AgentState: The updated state of the agent with the filtered URLs stored in state["AIdf"].

    """
    # filter to URL not previously seen
    AIdf = pd.DataFrame(state["AIdf"])

    AIdf = filter_unseen_urls_db(AIdf, before_date=state["before_date"])

    if len(AIdf) == 0:
        log("No new URLs, returning")
        return state

    # dedupe identical headlines
    # filter similar titles differing by type of quote or something
    AIdf['title'] = AIdf['title'].apply(unicode_to_ascii)
    AIdf['title_clean'] = AIdf['title'].map(lambda s: "".join(s.split()))
    AIdf = AIdf.sort_values("src") \
        .groupby("title_clean") \
        .first() \
        .reset_index(drop=True) \
        .drop(columns=['id']) \
        .reset_index() \
        .rename(columns={'index': 'id'})
    log(f"Found {len(AIdf)} unique new headlines")

    # structured response format
    json_schema = {
        "name": "json_schema",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "isai_array": {
                    "type": "array",
                    "items": {
                        "type": "object",
                                "properties": {
                                    "id": {
                                        "type": "number"
                                    },
                                    "isAI": {
                                        "type": "boolean"
                                    }
                                },
                        "required": ["id", "isAI"],
                        "additionalProperties": False
                    }
                }
            },
            "required": ["isai_array"],
            "additionalProperties": False
        }
    }

    # filter AI-related headlines using a prompt
    pages = paginate_df(AIdf)
    enriched_urls = asyncio.run(fetch_pages(pages, prompt=FILTER_PROMPT, json_schema=json_schema))
    filter_df = pd.DataFrame(enriched_urls)
    
    try:  # for idempotency
        AIdf = AIdf.drop(columns=['isAI'])
    except:
        pass

    # merge returned df with isAI column into original df on id column
    AIdf = pd.merge(AIdf, filter_df, on="id", how="outer")
    log(AIdf.columns)
    # set hostname based on actualurl
    AIdf['actual_url'] = AIdf['url']    # ideally resolve redirects but Google News blocks
    AIdf['hostname']=AIdf['actual_url'].apply(lambda url: urlparse(url).netloc)

    # update SQLite database with all seen URLs (we are doing this using url and ignoring redirects)
    log(f"Inserting {len(AIdf)} URLs into {SQLITE_DB}")
    conn = sqlite3.connect(SQLITE_DB)
    cursor = conn.cursor()
    for row in AIdf.itertuples():
        insert_article(conn, cursor, row.src, row.hostname, row.title,
                       row.url, row.actual_url, row.isAI, datetime.now().date())

    # keep headlines that are related to AI
    AIdf = AIdf.loc[AIdf["isAI"]==1] \
        .reset_index(drop=True)  \
        .reset_index()  \
        .drop(columns=["id"])  \
        .rename(columns={'index': 'id'})

    log(f"Found {len(AIdf)} AI headlines")

    # update actual URLs for Google News redirects
    # I think Google changed something so this no longer works, instead of a 301 redirct
    # get a javascript page that redirects. Also tomorrow we might see different URLs for same stories
    # AIdf = get_google_news_redirects(AIdf)

    conn = sqlite3.connect('articles.db')
    query = "select * from sites"
    sites_df = pd.read_sql_query(query, conn)
    sites_dict = {row.hostname:row.site_name for row in sites_df.itertuples()}
    conn.close()

    # get clean site_name
    AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, hostname))

    # if any missing clean site names, populate them using OpenAI
    missing_site_names =  len(AIdf.loc[AIdf['site_name']==""])
    if missing_site_names:
        log(f"Asking OpenAI for {missing_site_names} missing site names")
        responses = asyncio.run(fetch_missing_site_names(AIdf))
        # update site_dict from responses
        new_urls = []
        for r in responses:
            if r['url'].startswith('https://'):
                r['url'] = r['url'][8:]
            new_urls.append(r['url'])
            sites_dict[r['url']] = r['site_name']
            log(f"Looked up {r['url']} -> {r['site_name']}")
        # update sites table with new names
        for url in new_urls:
            sqlstr = "INSERT OR IGNORE INTO sites (hostname, site_name) VALUES (?, ?);"
            log(f"Updated {url} -> {sites_dict[url]}")
            conn.execute(sqlstr, (url, sites_dict[url]))
            conn.commit()
        # reapply to AIdf with updated sites
        AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, hostname))
    else:
        log("No missing site names")
  
    # drop banned slop sites

    AIdf = AIdf.loc[~AIdf["hostname"].str.lower().isin(HOSTNAME_SKIPLIST)]
    AIdf = AIdf.loc[~AIdf["site_name"].str.lower().isin(SITE_NAME_SKIPLIST)]
    
    state["AIdf"] = AIdf.to_dict()
    return state


if DEBUG:
    test_state["before_date"] = before_date
    _ = fn_filter_urls(test_state)
    display(pd.DataFrame(test_state["AIdf"]).groupby('src').count()[['id']])
    display(list(pd.DataFrame(test_state["AIdf"]).columns))



2024-10-02 18:16:00,932 - AInewsbot - INFO - Querying SQLite with where_clause: 
2024-10-02 18:16:01,537 - AInewsbot - INFO - URLs in orig_df: 913
2024-10-02 18:16:01,539 - AInewsbot - INFO - Existing URLs in DB: 154386
2024-10-02 18:16:01,565 - AInewsbot - INFO - New URLs in df filtered by URL: 277
2024-10-02 18:16:01,645 - AInewsbot - INFO - Existing src+title: 11
2024-10-02 18:16:01,645 - AInewsbot - INFO - New URLs in df filtered by src+title: 266
2024-10-02 18:16:01,655 - AInewsbot - INFO - Found 261 unique new headlines
2024-10-02 18:16:02,002 - AInewsbot - INFO - Applying prompt to 6 pages using gpt-4o-mini
2024-10-02 18:16:02,003 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:16:02,004 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:16:02,005 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:16:02,005 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:16:02,005 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:16:02,005 - AInewsbot - INFO - sent 11 items 
2024-10-02 18:16:0

,id
src,
Ars Technica,1
Bloomberg Tech,1
Business Insider,9
FT Tech,5
Feedly AI,44
Google News,22
Hacker News,3
Hacker News 2,2
HackerNoon,8


['id', 'src', 'title', 'url', 'isAI', 'actual_url', 'hostname', 'site_name']

In [15]:
# Function for topic extraction

def clean_topics(row, lcategories):
    """
    Cleans the extracted_topics and assigned_topics by removing certain common topics and combining them into a single list.

    Args:
        row (pandas.Series): The row containing the extracted_topics and assigned_topics.
        lcategories (set): The set of lowercase categories.

    Returns:
        list: The cleaned and combined list of topics.
    """
    extracted_topics = [x.title() for x in row.extracted_topics if x.lower() not in {"technology", "ai", "artificial intelligence"}]
    assigned_topics = [x.title() for x in row.assigned_topics if x.lower() in lcategories]
    combined = sorted(list(set(extracted_topics + assigned_topics)))
    combined = [s.replace("Ai", "AI") for s in combined]
    combined = [s.replace("Genai", "Gen AI") for s in combined]
    combined = [s.replace("Openai", "OpenAI") for s in combined]

    return combined


async def do_cat(AIdf, categories):
    """
    Sends a prompt to ChatGPT to select topics for the title for each row in AIdf
    which match the topics in categories.

    Args:
        AIdf (pandas.DataFrame): The DataFrame containing the headlines.
        categories (list): The list of topics to match with the headlines.

    Returns:
        dict: A dictionary where the keys are the row IDs and the values are lists
        of selected topics for each headline.

    """

    catdict = {}
    async with aiohttp.ClientSession() as session:
        for i, row in enumerate(AIdf.itertuples()):
            tasks = []
            log(f"Categorizing headline {row.id+1} of {len(AIdf)}")
            h = row.title
            log(h)
            for c in categories:
                task = asyncio.create_task(categorize_headline(h, c, session))
                tasks.append(task)
            responses = await asyncio.gather(*tasks)
            catdict[row.id] = [item for sublist in responses for item in sublist]
            log(str(catdict[row.id]))

    return catdict


def fn_topic_analysis(state: AgentState) -> AgentState:

    """
    Extracts and selects topics for each headline in the state['AIdf'] dataframe, scrubs them, and stores them back in the dataframe.

    Args:
        state (AgentState): The current state of the agent.

    Returns:
        AgentState: The updated state of the agent with the extracted and selected topics stored in state['AIdf'].
    """
    AIdf = pd.DataFrame(state["AIdf"])
    pages = paginate_df(AIdf)
    # apply topic extraction prompt to AI headlines
    log("start free-form topic extraction")
    json_schema = {
        "name": "extracted_topics",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "extracted_topics": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "id": {
                                "type": "number",
                            },
                            "topics": {
                                "type": "array",
                                "items": {
                                    "type": "string",
                                },
                            },
                        },
                        "required": ["id", "topics"],
                        "additionalProperties": False,
                    }
                }
            },
            "required": ["extracted_topics"],
            "additionalProperties": False,
        }
    }
    response = asyncio.run(fetch_pages(pages, prompt=TOPIC_PROMPT, json_schema=json_schema))
    topic_df = pd.DataFrame(response)
    topic_df = topic_df.rename(columns={'topics': 'extracted_topics'})
    log(f"{len(topic_df)} free-form topics extracted")
    all_topics = [item.lower() for row in topic_df.itertuples() for item in row.extracted_topics]
    item_counts = Counter(all_topics)
    filtered_topics = [item for item in item_counts if item_counts[item] >= 2 and item not in {'technology', 'ai', 'artificial intelligence'}]

    categories = sorted(CANONICAL_TOPICS)
    # use categories that are canonical or show up twice in freeform
    lcategories = set([c.lower() for c in categories] + [c.lower() for c in filtered_topics])
    # new topics
    log([c for c in filtered_topics if c not in categories])

    catdict = asyncio.run(categorize_headline(AIdf, categories=categories))
    topic_df['assigned_topics'] = topic_df['id'].apply(lambda id: catdict.get(id, ""))

    topic_df["topics"] = topic_df.apply(lambda t: clean_topics(t, lcategories), axis=1)
    topic_df["topic_str"] = topic_df.apply(lambda row: ", ".join(row.topics), axis=1)

    try: # for idempotency
        AIdf = AIdf.drop(columns=['topic_str', 'title_topic_str'])
    except:
        pass

    AIdf = pd.merge(AIdf, topic_df[["id", "topic_str"]], on="id", how="outer")
    AIdf['title_topic_str'] = AIdf.apply(lambda row: f'{row.title} (Topics: {row.topic_str})', axis=1)

    state["AIdf"] = AIdf.to_dict()
    return state

if DEBUG:
    _ = fn_topic_analysis(test_state)


2024-10-02 18:16:12,594 - AInewsbot - INFO - start free-form topic extraction
2024-10-02 18:16:12,594 - AInewsbot - INFO - Applying prompt to 3 pages using gpt-4o-mini
2024-10-02 18:16:12,595 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:16:12,595 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:16:12,595 - AInewsbot - INFO - sent 7 items 
2024-10-02 18:16:15,270 - AInewsbot - INFO - got dict with 7 items 
2024-10-02 18:16:25,511 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:16:29,047 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:16:29,050 - AInewsbot - INFO - Processed 107 responses.
2024-10-02 18:16:29,052 - AInewsbot - INFO - 107 free-form topics extracted
2024-10-02 18:16:29,054 - AInewsbot - INFO - ['economics', 'generative ai', 'coding', 'startup', 'poolside', 'funding', 'ebay', 'nvidia', 'podcast', 'content creation', 'accenture', 'partnership', 'training', 'enterprise', 'business group', 'employees', 'google', 'language models', 'chatbots', 'amazo

2024-10-02 18:17:54,592 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:17:54,594 - AInewsbot - INFO - Processed 107 responses.
2024-10-02 18:17:54,595 - AInewsbot - INFO - Autonomous vehicles, topic 12 of 154
2024-10-02 18:17:54,595 - AInewsbot - INFO - Applying prompt to 3 pages using gpt-4o-mini
2024-10-02 18:17:54,597 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:17:54,597 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:17:54,598 - AInewsbot - INFO - sent 7 items 
2024-10-02 18:17:56,439 - AInewsbot - INFO - got dict with 7 items 
2024-10-02 18:18:00,651 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:18:08,627 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:18:08,630 - AInewsbot - INFO - Processed 107 responses.
2024-10-02 18:18:08,631 - AInewsbot - INFO - Baidu, topic 13 of 154
2024-10-02 18:18:08,632 - AInewsbot - INFO - Applying prompt to 3 pages using gpt-4o-mini
2024-10-02 18:18:08,633 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:18

2024-10-02 18:19:39,568 - AInewsbot - INFO - Applying prompt to 3 pages using gpt-4o-mini
2024-10-02 18:19:39,569 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:19:39,570 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:19:39,571 - AInewsbot - INFO - sent 7 items 
2024-10-02 18:19:41,170 - AInewsbot - INFO - got dict with 7 items 
2024-10-02 18:19:44,780 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:19:46,832 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:19:46,834 - AInewsbot - INFO - Processed 107 responses.
2024-10-02 18:19:46,836 - AInewsbot - INFO - Code assistants, topic 26 of 154
2024-10-02 18:19:46,838 - AInewsbot - INFO - Applying prompt to 3 pages using gpt-4o-mini
2024-10-02 18:19:46,841 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:19:46,842 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:19:46,843 - AInewsbot - INFO - sent 7 items 
2024-10-02 18:19:48,634 - AInewsbot - INFO - got dict with 7 items 
2024-10-02 18:19:53,331 - AInewsbot - IN

2024-10-02 18:21:18,033 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:21:18,034 - AInewsbot - INFO - sent 7 items 
2024-10-02 18:21:19,715 - AInewsbot - INFO - got dict with 7 items 
2024-10-02 18:21:24,969 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:21:25,000 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:21:25,002 - AInewsbot - INFO - Processed 107 responses.
2024-10-02 18:21:25,003 - AInewsbot - INFO - Demis Hassabis, topic 39 of 154
2024-10-02 18:21:25,004 - AInewsbot - INFO - Applying prompt to 3 pages using gpt-4o-mini
2024-10-02 18:21:25,005 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:21:25,007 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:21:25,008 - AInewsbot - INFO - sent 7 items 
2024-10-02 18:21:26,471 - AInewsbot - INFO - got dict with 7 items 
2024-10-02 18:21:31,796 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:21:33,230 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:21:33,233 - AInewsbot - INFO - Processe

2024-10-02 18:23:03,055 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:23:05,905 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:23:05,908 - AInewsbot - INFO - Processed 107 responses.
2024-10-02 18:23:05,908 - AInewsbot - INFO - Finance, topic 52 of 154
2024-10-02 18:23:05,910 - AInewsbot - INFO - Applying prompt to 3 pages using gpt-4o-mini
2024-10-02 18:23:05,910 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:23:05,912 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:23:05,913 - AInewsbot - INFO - sent 7 items 
2024-10-02 18:23:09,478 - AInewsbot - INFO - got dict with 7 items 
2024-10-02 18:23:11,128 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:23:11,131 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:23:11,138 - AInewsbot - INFO - Processed 107 responses.
2024-10-02 18:23:11,139 - AInewsbot - INFO - Fintech, topic 53 of 154
2024-10-02 18:23:11,140 - AInewsbot - INFO - Applying prompt to 3 pages using gpt-4o-mini
2024-10-02 18:23:

...

2024-10-02 18:28:15,684 - AInewsbot - INFO - sent 7 items 
2024-10-02 18:28:16,799 - AInewsbot - INFO - got dict with 7 items 
2024-10-02 18:28:21,412 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:28:21,413 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:28:21,415 - AInewsbot - INFO - Processed 107 responses.
2024-10-02 18:28:21,415 - AInewsbot - INFO - Music, topic 92 of 154
2024-10-02 18:28:21,416 - AInewsbot - INFO - Applying prompt to 3 pages using gpt-4o-mini
2024-10-02 18:28:21,418 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:28:21,419 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:28:21,420 - AInewsbot - INFO - sent 7 items 
2024-10-02 18:28:22,845 - AInewsbot - INFO - got dict with 7 items 
2024-10-02 18:28:27,152 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:28:28,679 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:28:28,680 - AInewsbot - INFO - Processed 107 responses.
2024-10-02 18:28:28,681 - AInewsbot - INFO - Mustafa

2024-10-02 18:30:06,395 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:30:06,397 - AInewsbot - INFO - Processed 107 responses.
2024-10-02 18:30:06,398 - AInewsbot - INFO - Privacy, topic 105 of 154
2024-10-02 18:30:06,399 - AInewsbot - INFO - Applying prompt to 3 pages using gpt-4o-mini
2024-10-02 18:30:06,400 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:30:06,401 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:30:06,402 - AInewsbot - INFO - sent 7 items 
2024-10-02 18:30:07,817 - AInewsbot - INFO - got dict with 7 items 
2024-10-02 18:30:12,877 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:30:12,940 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:30:12,943 - AInewsbot - INFO - Processed 107 responses.
2024-10-02 18:30:12,944 - AInewsbot - INFO - Privacy & Surveillance, topic 106 of 154
2024-10-02 18:30:12,945 - AInewsbot - INFO - Applying prompt to 3 pages using gpt-4o-mini
2024-10-02 18:30:12,947 - AInewsbot - INFO - sent 50 items 
2024-10-0

2024-10-02 18:31:53,493 - AInewsbot - INFO - SAP, topic 118 of 154
2024-10-02 18:31:53,494 - AInewsbot - INFO - Applying prompt to 3 pages using gpt-4o-mini
2024-10-02 18:31:53,496 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:31:53,497 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:31:53,498 - AInewsbot - INFO - sent 7 items 
2024-10-02 18:31:55,201 - AInewsbot - INFO - got dict with 7 items 
2024-10-02 18:32:01,843 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:32:02,399 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:32:02,401 - AInewsbot - INFO - Processed 107 responses.
2024-10-02 18:32:02,401 - AInewsbot - INFO - Safety and Alignment, topic 119 of 154
2024-10-02 18:32:02,402 - AInewsbot - INFO - Applying prompt to 3 pages using gpt-4o-mini
2024-10-02 18:32:02,403 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:32:02,405 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:32:02,406 - AInewsbot - INFO - sent 7 items 
2024-10-02 18:32:03,647 - AInewsbot

2024-10-02 18:33:40,813 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:33:40,814 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:33:40,816 - AInewsbot - INFO - sent 7 items 
2024-10-02 18:33:41,959 - AInewsbot - INFO - got dict with 7 items 
2024-10-02 18:33:45,765 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:33:48,697 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:33:48,701 - AInewsbot - INFO - Processed 107 responses.
2024-10-02 18:33:48,702 - AInewsbot - INFO - Sports, topic 132 of 154
2024-10-02 18:33:48,703 - AInewsbot - INFO - Applying prompt to 3 pages using gpt-4o-mini
2024-10-02 18:33:48,705 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:33:48,706 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:33:48,707 - AInewsbot - INFO - sent 7 items 
2024-10-02 18:33:50,231 - AInewsbot - INFO - got dict with 7 items 
2024-10-02 18:34:00,267 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:34:00,412 - AInewsbot - INFO - got dict with 50 items 


2024-10-02 18:35:36,628 - AInewsbot - INFO - got dict with 7 items 
2024-10-02 18:35:43,592 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:35:43,594 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:35:43,596 - AInewsbot - INFO - Processed 107 responses.
2024-10-02 18:35:43,596 - AInewsbot - INFO - Transportation, topic 145 of 154
2024-10-02 18:35:43,597 - AInewsbot - INFO - Applying prompt to 3 pages using gpt-4o-mini
2024-10-02 18:35:43,598 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:35:43,599 - AInewsbot - INFO - sent 50 items 
2024-10-02 18:35:43,599 - AInewsbot - INFO - sent 7 items 
2024-10-02 18:35:45,538 - AInewsbot - INFO - got dict with 7 items 
2024-10-02 18:35:50,760 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:35:53,524 - AInewsbot - INFO - got dict with 50 items 
2024-10-02 18:35:53,527 - AInewsbot - INFO - Processed 107 responses.
2024-10-02 18:35:53,527 - AInewsbot - INFO - Travel, topic 146 of 154
2024-10-02 18:35:53,528 - AInew

In [16]:
async def write_topic_name(session, topic_list_str, max_retries=3, model=LOWCOST_MODEL):
    """
    Generates a name for a cluster based on a list of headline topics.

    Parameters:
    session (aiohttp.ClientSession): The client session for making async HTTP requests.
    topic_list_str (str): A string containing the list of headline topics.
    max_retries (int, optional): The maximum number of retries in case of an error. Defaults to 3.
    model (str, optional): The model to use for generating the topic name. Defaults to LOWCOST_MODEL.

    Returns:
    dict: A dictionary containing the generated topic name.

    Example Usage:
    title_topic_str_list = "Headline 1 (Topic: Topic 1)\n\nHeadline 2 (Topic: Topic 2)"
    result = await write_topic_name(session, title_topic_str_list)
    print(result)

    Output:
    {"topic_title": "Generated Topic Name"}
    ```
    """
    TOPIC_WRITER_PROMPT = f"""
You are a topic writing assistant. I will provide a list of headlines with extracted topics in parentheses.
Your task is to propose a name for a topic that very simply, clearly and accurately captures all the provided
headlines in less than 7 words. You will output a JSON object with the key "topic_title".

Example Input:
In the latest issue of Caixins weekly magazine: CATL Bets on 'Skateboard Chassis' and Battery Swaps to Dispell Market Concerns (powered by AI) (Topics: Battery Swaps, Catl, China, Market Concerns, Skateboard Chassis)

AI, cheap EVs, future Chevy  the week (Topics: Chevy, Evs)

Electric Vehicles and AI: Driving the Consumer & World Forward (Topics: Consumer, Electric Vehicles, Technology)

Example Output:
{{"topic_title": "Electric Vehicles"}}

Task
Propose the name for the overall topic based on the following provided headlines and individual topics:

{topic_list_str}
"""

    for i in range(max_retries):
        try:
            messages=[
                      {"role": "user", "content": TOPIC_WRITER_PROMPT
                      }]

            payload = {"model":  model,
                       "response_format": {"type": "json_object"},
                       "messages": messages,
                       "temperature": 0
                       }
#             print(topic_list_str)
            response = asyncio.run(fetch_openai(session, payload))
            response_dict = json.loads(response["choices"][0]["message"]["content"])
            return response_dict

            break
        except Exception as exc:
            log(f"Error: {exc}")

    return {}


async def afn_topic_clusters(state: AgentState) -> AgentState:
    """
    Fetches embeddings for the headlines, creates clusters of similar articles using DBSCAN, and sorts
    using the clusters and a traveling salesman shortest traversal in embedding space.

    Parameters:
    state (AgentState): The state of the agent.

    Returns:
    AgentState: The updated state of the agent.

    """
    AIdf = pd.DataFrame(state["AIdf"])

    log(f"Fetching embeddings for {len(AIdf)} headlines")
    embedding_model = 'text-embedding-3-large'
    client = OpenAI()
    response = client.embeddings.create(input=AIdf['title_topic_str'].tolist(),
                                        model=embedding_model)
    embedding_df = pd.DataFrame([e.model_dump()['embedding'] for e in response.data])

    # greedy traveling salesman sort
    log(f"Sort with nearest_neighbor_sort sort")
    sorted_indices = nearest_neighbor_sort(embedding_df)
    AIdf['sort_order'] = sorted_indices

    # do dimensionality reduction on embedding_df and cluster analysis
    log(f"Perform dimensionality reduction")
    with open("reducer.pkl", 'rb') as file:
        # Load the model from the file
        reducer = pickle.load(file)
    reduced_data = reducer.transform(embedding_df)
    log(f"Cluster with DBSCAN")
    dbscan = DBSCAN(eps=0.4, min_samples=3)  # Adjust eps and min_samples as needed
    AIdf['cluster'] = dbscan.fit_predict(reduced_data)
    AIdf.loc[AIdf['cluster'] == -1, 'cluster'] = 999

    # sort first by clusters found by DBSCAN, then by semantic ordering
    AIdf = AIdf.sort_values(['cluster', 'sort_order']) \
        .reset_index(drop=True) \
        .reset_index() \
        .drop(columns=["id"]) \
        .rename(columns={'index': 'id'})

    # show clusters
    cluster_topics = []
    with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
        async with aiohttp.ClientSession() as session:
            for i in range(30):
                tmpdf = AIdf.loc[AIdf['cluster']==i][["id", "title_topic_str"]]
                if len(tmpdf) ==0:
                    break
                display(tmpdf)
                title_topic_str_list = ("\n\n".join(tmpdf['title_topic_str'].to_list()))
                cluster_topic = await write_topic_name(session, title_topic_str_list)
                cluster_topic = cluster_topic['topic_title']
                cluster_topics.append(cluster_topic)
                log(f"I dub this cluster: {cluster_topic}")
    state["cluster_topics"] = cluster_topics
    AIdf["cluster_name"] = AIdf['cluster'].apply(lambda i: cluster_topics[i] if i < len(cluster_topics) else "")
    state["AIdf"] = AIdf.to_dict()
    return state

# TODO: could add a quality rating for stories based on site reputation, length, complexity of story
# could then add the quality rating to the summaries and tell the prompt to favor high-quality stories
# could put summaries into vector store and retrieve stories by topic. but then you will have to deal
# with duplicates across categories, ask the prompt to dedupe

def fn_topic_clusters(state: AgentState) -> AgentState:
    "call async afn_topic_clusters on state"
    asyncio.run(afn_topic_clusters(state))
    return state


if DEBUG:
    _ = fn_topic_clusters(test_state)


2024-10-02 18:37:02,488 - AInewsbot - INFO - Fetching embeddings for 107 headlines
2024-10-02 18:37:02,997 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-10-02 18:37:03,375 - AInewsbot - INFO - Sort with nearest_neighbor_sort sort
2024-10-02 18:37:03,400 - AInewsbot - INFO - Perform dimensionality reduction
2024-10-02 18:37:06,416 - AInewsbot - INFO - Cluster with DBSCAN


,id,title_topic_str
0,0,"For Nvidia, spatial AI and the omniverse entering physical world may be the next big thing (Topics: Big Tech, Computer Vision, Economics, Hardware, Jensen Huang, Nvidia, Omniverse, Science, Semiconductor Chips, Spatial AI, Virtual & Augmented Reality)"
1,1,"Nvidia Partners With Accenture to Boost Corporate AI Adoption (Topics: Accenture, Big Tech, Corporate AI, Gen AI, Jensen Huang, Nvidia, Partnership, Science, Semiconductor Chips)"
2,2,"Nvidia Teams Up With Accenture to Boost Corporate AI Use (Topics: Accenture, Big Tech, Collaboration, Corporate AI, Gen AI, Jensen Huang, Nvidia, Science, Semiconductor Chips)"
3,3,"Accenture forms new Nvidia business group focused on agentic AI adoption (Topics: Accenture, Adoption, Big Tech, Business Group, Deals, Economics, Hardware, Jensen Huang, Nvidia, Safety And Alignment, Science, Semiconductor Chips, Stocks)"
4,4,"Accenture and NVIDIA Partner to Train 30,000 Professionals to Scale Agentic AI for Enterprises (Topics: Accenture, Big Tech, Deals, Economics, Education, Enterprise, Hardware, Jensen Huang, Jobs & Careerslabor Market, Nvidia, Professionals, Safety And Alignment, Science, Semiconductor Chips, Stocks, Training)"
5,5,"Accenture says it will train 30,000 employees on Nvidia technology and start a new Nvidia Business Group focused on driving enterprise adoption of AI agents (Topics: Accenture, Big Tech, Business Group, Deals, Economics, Education, Employees, Hardware, Jensen Huang, Jobs & Careerslabor Market, Nvidia, Safety And Alignment, Science, Semiconductor Chips, Stocks, Training, Virtual Assistants)"


2024-10-02 18:37:07,154 - AInewsbot - INFO - I dub this cluster: Nvidia and Accenture in AI Innovation


,id,title_topic_str
6,6,"Character.AI CEO Dominic Perella says the startup has largely abandoned building AI models after Google poached its founders, focusing on its chatbots instead (Topics: AI Models, Big Tech, Character.AI, Chatbots, Chatgpt, Deals, Dominic Perella, Google, Language Models, Mergers And Acquisitions, Science, Sergey Brin, Virtual Assistants)"
7,7,"Character.ai abandons making AI models after $2.7bn Google deal (Topics: AI Models, Big Tech, Character.AI, Chatbots, Chatgpt, Deal, Deals, Finance, Funding, Google, Language Models, Larry Page, Mergers And Acquisitions, Science, Sergey Brin, Virtual Assistants)"
8,8,"After Google's $2.7B Acquisition Of Founders And Staff, This AI Startup Abandons Large Language Model Plans And Shifts Focus Away From Chatbots (Topics: Acquisition, Big Tech, Chatbots, Chatgpt, Deals, Google, Language Models, Mergers And Acquisitions, Science, Sergey Brin, Startup)"


2024-10-02 18:37:07,869 - AInewsbot - INFO - I dub this cluster: Character.AI and Google's Impact on AI


...

2024-10-02 18:37:11,387 - AInewsbot - INFO - I dub this cluster: AI in Software Development Funding


,id,title_topic_str
39,39,"How to Generate an AI Podcast Using Googles NotebookLM (Topics: Authors & Writing, Entertainment, Gen AI, Google, Language Models, Music, Notebooklm, Podcast, Science, Speech Recognition & Synthesis, Streaming, Tv & Film & Movies)"
40,40,"How to Generate an AI Podcast Using Google's NotebookLM (Topics: Authors & Writing, Entertainment, Gen AI, Google, Language Models, Music, Notebooklm, Podcast, Science, Speech Recognition & Synthesis, Streaming, Tv & Film & Movies)"
41,41,"Google's AI generated a 'podcast' from one of my articles and it's incredibly convincing and creepy just how well it can mimic humans talking (Topics: Authors & Writing, Big Tech, Content Creation, Entertainment, Google, Language Models, Mimicry, Music, Opinion, Podcast, Privacy, Privacy & Surveillance, Review, Sergey Brin, Society & Culture, Speech Recognition & Synthesis, Streaming, Tv & Film & Movies)"
42,42,"Google's AI Turns the Words ""Fart"" and ""Poop"" Written 1,000 Times Into an Entire Podcast (Topics: Big Tech, Content Creation, Entertainment, Food & Drink, Google, Music, Podcast, Sergey Brin, Society & Culture, Streaming, Tv & Film & Movies)"


2024-10-02 18:37:12,220 - AInewsbot - INFO - I dub this cluster: AI-Generated Podcasts and Content Creation


,id,title_topic_str
43,43,"The US Army Is Testing Killer Robot Dogs With AI-Powered Rifles in the Middle East (Topics: AI Doom, Drones, Ethics, Gen AI, Governance, Hardware, Job Automation, Military, Policy And Regulation, Politics, Rifles, Robot Dogs, Robots, Safety And Alignment, Science, Testing, Transportation, Us Army)"
44,44,"The Army is Testing Robot Dogs With AI Guns in the Middle East (Topics: AI Doom, Drones, Ethics, Gen AI, Governance, Hardware, Job Automation, Military, Military Technology, Policy And Regulation, Politics, Robot Dogs, Robots, Safety And Alignment, Science, Testing, Transportation, Us Army)"
45,45,"US Army Testing Robot Dogs With AI-Enabled Rifles in Middle East (Topics: AI Doom, Drones, Ethics, Gen AI, Governance, Hardware, Job Automation, Military, Policy And Regulation, Politics, Rifles, Robot Dogs, Robots, Safety And Alignment, Science, Testing, Transportation, Us Army)"


2024-10-02 18:37:12,989 - AInewsbot - INFO - I dub this cluster: AI and Military Robotics


In [17]:
# scrape individual pages
def fn_download_pages(state: AgentState) -> AgentState:
    """
    Uses several Selenium browser sessions to download all the pages referenced in the
    state["AIdf"] DataFrame and store their pathnames.

    Args:
        state (AgentState): The current state of the agent.

    Returns:
        AgentState: The updated state of the agent with the downloaded pages' pathnames stored in the `state["AIdf"]` DataFrame.
    """
    log("Queuing URLs for scraping")
    AIdf = pd.DataFrame(state['AIdf'])
    queue = multiprocessing.Queue()

    count = 0
    for row in AIdf.itertuples():
#         if row.cluster < 999:
        queue.put((row.id, row.url, row.title))
        count +=1
    # scrape urls in queue asynchronously
    num_browsers = 4

    callable = process_url_queue_factory(queue)

    log(f"fetching {count} pages using {num_browsers} browsers")
    saved_pages = launch_drivers(num_browsers, callable)

    pages_df = pd.DataFrame(saved_pages)
    if len(pages_df):
        pages_df.columns = ['id', 'url', 'title', 'path']

        try: # for idempotency
            AIdf = AIdf.drop(columns=['path'])
        except:
            pass        
        AIdf = pd.merge(AIdf, pages_df[["id", "path"]], on='id', how="inner")
    state["AIdf"] = AIdf.to_dict()
    return state


if DEBUG:
    _ = fn_download_pages(test_state)



2024-10-02 18:37:13,028 - AInewsbot - INFO - Queuing URLs for scraping
2024-10-02 18:37:13,033 - AInewsbot - INFO - fetching 107 pages using 4 browsers
2024-10-02 18:37:13,034 - AInewsbot - INFO - get_driver - 25581 Initializing webdriver
2024-10-02 18:37:13,036 - AInewsbot - INFO - get_driver - 25581 Initializing webdriver
2024-10-02 18:37:13,036 - AInewsbot - INFO - get_driver - 25581 Initializing webdriver
2024-10-02 18:37:13,037 - AInewsbot - INFO - get_driver - 25581 Initializing webdriver
2024-10-02 18:37:29,167 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-10-02 18:37:29,167 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-10-02 18:37:29,168 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-10-02 18:37:29,168 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-10-02 18:37:29,168 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-10-02 18:37:29,168 - AInewsbot - INFO - get_driver - In

2024-10-02 18:38:44,214 - AInewsbot - INFO - Processing https://news.google.com/read/CBMi3gFBVV95cUxOSzAxeHAtV2Q3Vl9Ec1FTb1VNMHVUQmtYdHdUdG5ETWt2dENobEhXTEN0QnpWUEE2OHppdzlnZXQyVnBwTzVWbGFiZFphTU5oZ0ZkOS1aUWZjMENmd1ItcHRjc2FqSXM0SlF6NmZjS3VzTHdCeTFvZkpPb0ZULWtYNU9nZENldl9mZXU1S2gzMW9hdXpaTXdDcGNHUWY5TFF2UDcycnBxcGZQSzZxbFdZajlxQW9NY0FObmZjNkRUcjhMYWhMQ2ZiTUdHMFZsbHBmRDJKem8wZGotaEdOVXc
2024-10-02 18:38:44,215 - AInewsbot - INFO - get_url(https://news.google.com/read/CBMi3gFBVV95cUxOSzAxeHAtV2Q3Vl9Ec1FTb1VNMHVUQmtYdHdUdG5ETWt2dENobEhXTEN0QnpWUEE2OHppdzlnZXQyVnBwTzVWbGFiZFphTU5oZ0ZkOS1aUWZjMENmd1ItcHRjc2FqSXM0SlF6NmZjS3VzTHdCeTFvZkpPb0ZULWtYNU9nZENldl9mZXU1S2gzMW9hdXpaTXdDcGNHUWY5TFF2UDcycnBxcGZQSzZxbFdZajlxQW9NY0FObmZjNkRUcjhMYWhMQ2ZiTUdHMFZsbHBmRDJKem8wZGotaEdOVXc) - starting get_url https://news.google.com/read/CBMi3gFBVV95cUxOSzAxeHAtV2Q3Vl9Ec1FTb1VNMHVUQmtYdHdUdG5ETWt2dENobEhXTEN0QnpWUEE2OHppdzlnZXQyVnBwTzVWbGFiZFphTU5oZ0ZkOS1aUWZjMENmd1ItcHRjc2FqSXM0SlF6NmZjS3VzTHdCeTFvZkpPb0ZULWtY

2024-10-02 18:39:06,279 - AInewsbot - INFO - get_url(https://www.fastcompany.com/91201228/california-spiked-a-landmark-ai-regulation-but-that-doesnt-mean-the-bill-is-going-away) - starting get_url https://www.fastcompany.com/91201228/california-spiked-a-landmark-ai-regulation-but-that-doesnt-mean-the-bill-is-going-away
2024-10-02 18:39:06,587 - AInewsbot - INFO - get_url(Apparate: Early-Exit Models for ML Latency and Throughput Optimization - Latency-Focused Adjustments) - Saving Apparate__Early-Exit_Models_for_ML_Latency_and_Throughput_Optimization_-_Latency-Focused_Adjustments_20241002_183906.html as utf-8
2024-10-02 18:39:06,588 - AInewsbot - INFO - Processing https://www.ft.com/content/6e460960-9530-40ac-8346-cd25b94f8f32
2024-10-02 18:39:06,590 - AInewsbot - INFO - get_url(https://www.ft.com/content/6e460960-9530-40ac-8346-cd25b94f8f32) - starting get_url https://www.ft.com/content/6e460960-9530-40ac-8346-cd25b94f8f32
2024-10-02 18:39:08,690 - AInewsbot - INFO - get_url(Apparate: 

2024-10-02 18:39:37,804 - AInewsbot - INFO - get_url(https://venturebeat.com/ai/openai-isnt-going-anywhere-raises-6-6b-at-157b-valuation/) - starting get_url https://venturebeat.com/ai/openai-isnt-going-anywhere-raises-6-6b-at-157b-valuation/
2024-10-02 18:39:38,789 - AInewsbot - INFO - get_url(OpenAI just closed a megaround. Here's how its funding stacks up to other Bay Area AI startups) - Saving OpenAI_just_closed_a_megaround__Here_s_how_its_funding_stacks_up_to_other_Bay_Area_AI_startups_20241002_183938.html as utf-8
2024-10-02 18:39:38,790 - AInewsbot - INFO - Processing https://news.google.com/read/CBMiogFBVV95cUxOcjMtZGtlZHVYRWVtR1ptTmJUWUtTRTJ6ejFkZzRoaDJTVkVsMlhOUkc4TVVCSm4tMWwxMzlrZTExLTVZUHAwYnY3ZzdPVWdhMEhEa2tnRnF3MlJjTk9xOHhwcV94YTdnY1Z1OHNpUm9TN2hxVFpzS0JMSGpqaXJDX19RU3EzR3JVWFhpak9rNTdVaWNXNWwwaXp6M0NwLU55aGc
2024-10-02 18:39:38,790 - AInewsbot - INFO - get_url(https://news.google.com/read/CBMiogFBVV95cUxOcjMtZGtlZHVYRWVtR1ptTmJUWUtTRTJ6ejFkZzRoaDJTVkVsMlhOUkc4TVVCSm4tMWw

2024-10-02 18:40:04,537 - AInewsbot - INFO - get_url(OpenAI completes deal that values company at $157B) - Saving OpenAI_completes_deal_that_values_company_at__157B_20241002_184004.html as utf-8
2024-10-02 18:40:04,539 - AInewsbot - INFO - Processing https://news.google.com/read/CBMi2wFBVV95cUxPakVZcGplWEs1SVlIeFIzNWdMZHU3dEhPQXFnVmdUcDMzSEFuT2gzRlZ6Y0xKeTFvVk5yVnoyYU5DOGZpYVhmbXNoNWtaT3MtYzRPVWlnakZwc3ZyT2ZVRkZYdi12NENvZDRzcVVUTmtpR3VIdmF4VmxlODdHQV9Fdlk2QkNFcFdaOGdaTU1JcXlKd0ZGTzJ0bjk0Y21BTy1WaElmM3VNb2FUTm5naTdGS3ZCQ1ZDTzVsbGVwNnhGRDFMYmg1ZEtkenNSYWRMQnFQc19USEN4QzRMbkk
2024-10-02 18:40:04,540 - AInewsbot - INFO - get_url(https://news.google.com/read/CBMi2wFBVV95cUxPakVZcGplWEs1SVlIeFIzNWdMZHU3dEhPQXFnVmdUcDMzSEFuT2gzRlZ6Y0xKeTFvVk5yVnoyYU5DOGZpYVhmbXNoNWtaT3MtYzRPVWlnakZwc3ZyT2ZVRkZYdi12NENvZDRzcVVUTmtpR3VIdmF4VmxlODdHQV9Fdlk2QkNFcFdaOGdaTU1JcXlKd0ZGTzJ0bjk0Y21BTy1WaElmM3VNb2FUTm5naTdGS3ZCQ1ZDTzVsbGVwNnhGRDFMYmg1ZEtkenNSYWRMQnFQc19USEN4QzRMbkk) - starting get_url https://news.googl

...

2024-10-02 18:41:02,085 - AInewsbot - INFO - get_url(Its worrying: there are still so many people out there who dont get AI) - Saving Its_worrying__there_are_still_so_many_people_out_there_who_dont_get_AI_20241002_184102.html as utf-8
2024-10-02 18:41:02,086 - AInewsbot - INFO - Processing https://futurism.com/the-byte/ai-programming-assistants-code-error
2024-10-02 18:41:02,086 - AInewsbot - INFO - get_url(https://futurism.com/the-byte/ai-programming-assistants-code-error) - starting get_url https://futurism.com/the-byte/ai-programming-assistants-code-error
2024-10-02 18:41:09,055 - AInewsbot - INFO - get_url(Improving Text-to-SQL with a Fine-Tuned 7B LLM for DB Interactions) - Saving Improving_Text-to-SQL_with_a_Fine-Tuned_7B_LLM_for_DB_Interactions_20241002_184109.html as utf-8
2024-10-02 18:41:09,057 - AInewsbot - INFO - Processing https://www.techradar.com/pro/security/this-ai-powered-malware-has-just-evolved-to-add-image-recognition
2024-10-02 18:41:09,057 - AInewsbot - INFO - ge

2024-10-02 18:41:43,703 - AInewsbot - INFO - get_url(OpenAI Raises Billions While Pledging to Work With U.S. and Allied Governments) - Saving OpenAI_Raises_Billions_While_Pledging_to_Work_With_U_S__and_Allied_Governments_20241002_184143.html as utf-8
2024-10-02 18:41:43,704 - AInewsbot - INFO - Processing https://nypost.com/2024/10/02/tech/facebook-instagram-users-outraged-over-ai-training-with-user-posts-f-k-right-off/
2024-10-02 18:41:43,704 - AInewsbot - INFO - get_url(https://nypost.com/2024/10/02/tech/facebook-instagram-users-outraged-over-ai-training-with-user-posts-f-k-right-off/) - starting get_url https://nypost.com/2024/10/02/tech/facebook-instagram-users-outraged-over-ai-training-with-user-posts-f-k-right-off/
2024-10-02 18:41:44,207 - AInewsbot - INFO - get_url(Boulder Police Department is using AI to write its crime reports, while some urge caution) - Saving Boulder_Police_Department_is_using_AI_to_write_its_crime_reports__while_some_urge_caution_20241002_184144.html as ut

2024-10-02 18:42:38,163 - AInewsbot - INFO - Processing https://news.google.com/read/CBMiqgFBVV95cUxPYnhkd2Q3eE5nUjl0NF9tTC1PTklENENrSW5tbHltMXF5anliMXM0ZWs1Q3hxcVBBaHdCX1ZHR3ZtS0F2Ymx5b2dQRWtEUEZ4TGpxaXBzWC1uU19VUEZhbkVMbEp1Y0dxQ2RTdGttZlU1N0pCMmlKOENaRWN3QUFNTUZLUjU4MTVac3RLdElFaEtZZDVFYUxhVTB4V0pETlBucGxkSFZBU1dMQQ
2024-10-02 18:42:38,168 - AInewsbot - INFO - get_url(https://news.google.com/read/CBMiqgFBVV95cUxPYnhkd2Q3eE5nUjl0NF9tTC1PTklENENrSW5tbHltMXF5anliMXM0ZWs1Q3hxcVBBaHdCX1ZHR3ZtS0F2Ymx5b2dQRWtEUEZ4TGpxaXBzWC1uU19VUEZhbkVMbEp1Y0dxQ2RTdGttZlU1N0pCMmlKOENaRWN3QUFNTUZLUjU4MTVac3RLdElFaEtZZDVFYUxhVTB4V0pETlBucGxkSFZBU1dMQQ) - starting get_url https://news.google.com/read/CBMiqgFBVV95cUxPYnhkd2Q3eE5nUjl0NF9tTC1PTklENENrSW5tbHltMXF5anliMXM0ZWs1Q3hxcVBBaHdCX1ZHR3ZtS0F2Ymx5b2dQRWtEUEZ4TGpxaXBzWC1uU19VUEZhbkVMbEp1Y0dxQ2RTdGttZlU1N0pCMmlKOENaRWN3QUFNTUZLUjU4MTVac3RLdElFaEtZZDVFYUxhVTB4V0pETlBucGxkSFZBU1dMQQ
2024-10-02 18:42:48,436 - AInewsbot - INFO - get_url(AMD GPU Inference) - Savin

2024-10-02 18:43:21,310 - AInewsbot - INFO - get_url(AI coding startup Poolside raises $500M from eBay, Nvidia and others) - Saving AI_coding_startup_Poolside_raises__500M_from_eBay__Nvidia_and_others_20241002_184321.html as utf-8
2024-10-02 18:43:21,317 - AInewsbot - INFO - Processing https://www.makeuseof.com/niche-bots-better-than-chatgpt/
2024-10-02 18:43:21,321 - AInewsbot - INFO - get_url(https://www.makeuseof.com/niche-bots-better-than-chatgpt/) - starting get_url https://www.makeuseof.com/niche-bots-better-than-chatgpt/
2024-10-02 18:43:32,023 - AInewsbot - INFO - get_url(Mark Zuckerberg Sparks Debate After Using AI To Illustrate His Young Daughter's Book) - Saving Mark_Zuckerberg_Sparks_Debate_After_Using_AI_To_Illustrate_His_Young_Daughter_s_Book_20241002_184332.html as utf-8
2024-10-02 18:43:32,026 - AInewsbot - INFO - Processing https://www.indiehackers.com/starting-up/the-ai-startup-drama-thats-damaging-y-combinator-s-reputation-GQKuTmpGV2uWOCoxtHBn
2024-10-02 18:43:32,027

2024-10-02 18:44:05,262 - AInewsbot - INFO - get_url(Tesla trounces shareholders who alleged Autopilot was all share-pumping liesIt's easy being ElonAI + ML3 hrs|2) - Saving Tesla_trounces_shareholders_who_alleged_Autopilot_was_all_share-pumping_liesIt_s_easy_being_ElonAI___ML3_hrs_2_20241002_184405.html as utf-8
2024-10-02 18:44:05,265 - AInewsbot - INFO - Processing https://www.reddit.com/r/ProgrammerHumor/comments/1fueur3/promptengineering/
2024-10-02 18:44:05,265 - AInewsbot - INFO - get_url(https://www.reddit.com/r/ProgrammerHumor/comments/1fueur3/promptengineering/) - starting get_url https://www.reddit.com/r/ProgrammerHumor/comments/1fueur3/promptengineering/
2024-10-02 18:44:08,320 - AInewsbot - INFO - get_url(Building A Good Archive Of The Internet Means AI Content Goes In The Trash) - Saving Building_A_Good_Archive_Of_The_Internet_Means_AI_Content_Goes_In_The_Trash_20241002_184408.html as utf-8
2024-10-02 18:44:08,322 - AInewsbot - INFO - Processing https://venturebeat.com/ai

In [18]:
# summarize individual pages

def fn_summarize_pages(state: AgentState) -> AgentState:
    """
    Reads all the articles, summarizes each one using a ChatGPT prompt, and sends an email with the summaries.

    Args:
        state (AgentState): The current state of the agent.

    Returns:
        AgentState: The updated state of the agent.

    """
    log("Starting summarize")
    AIdf = pd.DataFrame(state['AIdf'])
    responses = asyncio.run(fetch_all_summaries(AIdf))
    log(f"Received {len(responses)} summaries")
    response_dict = {}
    for i, response in responses:
        try:
            response_str = response["choices"][0]["message"]["content"]
            response_dict[i] = response_str
        except Exception as exc:
            print(exc)

    markdown_str = ''
    bullets = []

    for i, row in enumerate(AIdf.itertuples()):
        topics = []
        if row.cluster_name:
            topics.append(row.cluster_name)
        if row.topic_str:
            topics.append(row.topic_str)
        topic_str = ", ".join(topics)

        mdstr = f"[{i+1}. {row.title} - {row.site_name}]({row.actual_url})  \n\n {topic_str}  \n\n{response_dict[row.id]} \n\n"
        bullets.append(f"[{row.title} - {row.site_name}]({row.actual_url})\n\nTopics: {row.topic_str} \n\n{response_dict[row.id]}\n\n")
        display(Markdown(mdstr.replace("$","\\\\$")))
        markdown_str += mdstr

    state['bullets'] = bullets
    # Convert Markdown to HTML
    html_str = markdown.markdown(markdown_str, extensions=['extra'])
    # save bullets
    with open('bullets.md', 'w') as f:
        f.write(markdown_str)
    # send email
    log("Sending bullet points email")
    subject = f'AI news bullets {datetime.now().strftime("%H:%M:%S")}'
    send_gmail(subject, html_str)

    return state


if DEBUG:
    _ = fn_summarize_pages(test_state)



2024-10-02 18:46:24,087 - AInewsbot - INFO - Starting summarize
2024-10-02 18:46:24,142 - AInewsbot - INFO - fetch_all_summaries - Page title: For Nvidia, spatial AI and the ‘omniverse’ may be the next big thing
Social card title: For Nvidia, spatial AI and the ‘omniverse’ entering physical world may be the next big thing
Social card description: Spatial artificial intelligence and robotics is developing as we speak, and it's bringing the technological focus back into the physical world.

2024-10-02 18:46:24,306 - AInewsbot - INFO - fetch_all_summaries - Page title: Nvidia (NVDA) Partners With Accenture to Boost Corporate AI Adoption - Bloomberg
Social card title: Nvidia Partners With Accenture to Boost Corporate AI Adoption
Social card description: Nvidia Corp. has expanded a partnership with technology consultant Accenture as part of an effort to drive adoption of artificial intelligence within businesses and boost orders for the chipmaker’s products.

2024-10-02 18:46:24,397 - AInew

2024-10-02 18:46:25,474 - AInewsbot - INFO - fetch_all_summaries - Page title: California spiked a landmark AI regulation. But that doesn't mean the bill is going away - Fast Company
Social card title: California spiked a landmark AI regulation. But that doesn't mean the bill is going away
Social card description: State Senator Scott Wiener hasn’t ruled out the possibility of introducing the bill again next session.

2024-10-02 18:46:25,500 - AInewsbot - INFO - fetch_all_summaries - Page title: Subscribe to read

2024-10-02 18:46:25,536 - AInewsbot - INFO - fetch_all_summaries - Page title: What Gavin Newsom just did to the global AI debate - POLITICO
Social card title: What Gavin Newsom just did to the global AI debate

2024-10-02 18:46:25,568 - AInewsbot - INFO - fetch_all_summaries - Page title: OpenAI just raised $6.6 billion to build ever-larger AI models - The Verge
Social card title: OpenAI just raised $6.6 billion to build ever-larger AI models
Social card description: A histor

2024-10-02 18:46:26,471 - AInewsbot - INFO - fetch_all_summaries - Page title: Google's AI generated a 'podcast' from one of my articles and it's incredibly convincing and creepy just how well it can mimic humans talking | PC Gamer
Social card title: Google's AI generated a 'podcast' from one of my articles and it's incredibly convincing and creepy just how well it can mimic humans talking
Social card description: A podcast generated in minutes by a bot—and yet I've heard worse.

2024-10-02 18:46:26,524 - AInewsbot - INFO - fetch_all_summaries - Page title: Google's AI Turns the Words "Fart" and "Poop" Written 1,000 Times Into an Entire Podcast
Social card title: Google's AI Turns the Words "Fart" and "Poop" Written 1,000 Times Into an Entire Podcast
Social card description: A Reddit user forced Google's NotebookLM AI model to riff on a document that was literally full of crap for ten minutes straight.

2024-10-02 18:46:26,540 - AInewsbot - INFO - fetch_all_summaries - Page title: The 

2024-10-02 18:46:27,293 - AInewsbot - INFO - fetch_all_summaries - Page title: This AI-powered malware has evolved to add image recognition | TechRadar
Social card title: This AI-powered malware has evolved to add image recognition
Social card description: Information stealing gets a worrying upgrade

2024-10-02 18:46:27,335 - AInewsbot - INFO - fetch_all_summaries - Page title: The next iPhone SE may lose the home button, add Face ID and Apple Intelligence - The Verge
Social card title: The next iPhone SE may lose the home button, add Face ID and Apple Intelligence
Social card description: The next iPhone SE will be more than just a budget phone.

2024-10-02 18:46:27,369 - AInewsbot - INFO - fetch_all_summaries - Page title: French AI summit to focus on environmental impact of energy-hungry tech | Artificial intelligence (AI) | The Guardian
Social card title: French AI summit to focus on environmental impact of energy-hungry tech
Social card description: Event will push for greater tr

2024-10-02 18:46:28,782 - AInewsbot - INFO - fetch_all_summaries - Page title: Nvidia Stock: Accenture, Chipmaker Forge Artificial Intelligence Partnership | Investor's Business Daily
Social card title: Accenture, Chipmaker Nvidia Forge Artificial Intelligence Partnership
Social card description: Nvidia stock rose after the chipmaker and Accenture unveiled a partnership to drive corporate adoption of artificial intelligence.

2024-10-02 18:46:28,810 - AInewsbot - INFO - fetch_all_summaries - Page title: AI coding startup Poolside raises $500M from eBay, Nvidia, and others | TechCrunch
Social card title: AI coding startup Poolside raises $500M from eBay, Nvidia, and others | TechCrunch
Social card description: Poolside, a startup developing an AI coding assistant aimed at software engineers, has raised half a billion dollars in a Series B round.

2024-10-02 18:46:28,845 - AInewsbot - INFO - fetch_all_summaries - Page title: AI coding startup Poolside raises $500M from eBay, Nvidia, and 

...

[103. Building A Good Archive Of The Internet Means AI Content Goes In The Trash - worldcrunch.com](https://worldcrunch.com/tech-science/ai-internet-archives)  

 Archive, Content, Internet, Media, Opinion  

- The proliferation of AI-generated content threatens the preservation of valuable online information, necessitating a thoughtful approach to archiving that prioritizes true and useful data over low-effort materials.
- Costly data preservation efforts require significant resources and raise concerns about copyright compliance, making it essential to clarify who will manage and fund these initiatives.
- Government intervention is critical, as it has the potential to provide the funding and legal frameworks needed to effectively archive digital content and maintain public access to knowledge. 



[104. promptEngineering - Reddit](https://www.reddit.com/r/ProgrammerHumor/comments/1fueur3/promptengineering/)  

 Development, Language Models, Prompt Engineering  

- The text appears to be a brief discussion on a Reddit page about programming humor and comments related to structured data output in AI models.
- Users are sharing thoughts on the reliability of JSON outputs from AI, particularly referencing recent updates from OpenAI.
- A humorous comment about procrastination indicates a casual and light-hearted tone among participants in the discussion. 



[105. Pika 1.5 launches with physics-defying AI special effects - VentureBeat](https://venturebeat.com/ai/pika-1-5-launches-with-physics-defying-ai-special-effects/)  

 Gen AI, Pika 1.5, Products, Science, Special Effects  

- Pika 1.5 introduces "Pikaffects," which provide surreal special effects that manipulate imagery and video subjects in unexpected ways, including options like "explode," "melt," and "cake-ify."
- The updated platform enhances video generation with improved workflows, allowing users to create high-quality five-second clips featuring realistic movements and advanced cinematic shots while maintaining creative flexibility.
- While subscription prices remain the same, generating clips now requires more credits, and Pika encourages user feedback and experimentation through community challenges. 



[106. Uber upgrades platform with scalable GPU clusters - SiliconANGLE - SiliconANGLE](https://siliconangle.com/2024/10/02/uber-scalable-gpu-clusters-anyscale-raysummit/)  

 Big Tech, Gpu Clusters, Hardware, Intel, Platform, Products, Science, Semiconductor Chips, Transportation, Uber  

- Uber is enhancing its AI platform by implementing scalable GPU clusters with Anyscale Ray, allowing for over 20,000 model training jobs monthly.
- The company is evolving its Michelangelo AI platform to handle increasing data workloads and facilitating resource sharing through Kubernetes, thereby improving efficiency.
- Uber emphasizes the importance of open source in building its innovative AI ecosystem, focusing on responsible AI tools and the challenges of managing data across hybrid and multi-cloud environments. 



2024-10-02 18:46:34,371 - AInewsbot - INFO - Sending bullet points email


In [19]:
def fn_propose_cats(state: AgentState) -> AgentState:
    # ask chatgpt for top categories

    model = ChatOpenAI(
        model=MODEL,
        temperature=0.3,
        model_kwargs={"response_format": {"type": "json_object"}}
    )

    chain = ChatPromptTemplate.from_template("{p}") | model | SimpleJsonOutputParser()
    response = chain.invoke({ "p": TOP_CATEGORIES_PROMPT + "\n\n".join(state["bullets"])})
    suggested_categories = []
    for k, v in response.items():
        suggested_categories.extend(v)
    state["cluster_topics"] = list(set(state["cluster_topics"] + suggested_categories))
    state["cluster_topics"].sort()
    return state


if DEBUG:
    _ = fn_propose_cats(test_state)
    print("edit the following proposed topic list and update state['cluster_topics']:")
    print('"' + '",\n"'.join(test_state["cluster_topics"]) + '"')


2024-10-02 18:46:42,024 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


edit the following proposed topic list and update state['cluster_topics']:
"AI Regulation and Governance in California",
"AI and Military Robotics",
"AI assistants leaking workplace secrets",
"AI in Software Development Funding",
"AI startup drama at Y Combinator",
"AI-Generated Podcasts and Content Creation",
"AI-powered malware evolves with image recognition",
"Apparate ML optimization",
"California AI regulation veto",
"Cathie Wood's Ark invests in OpenAI",
"Character.AI and Google's Impact on AI",
"Character.AI shifts focus after Google deal",
"Crescendo raises $50M for AI customer service",
"Google's Gemini AI on Android",
"Intel updates AI Playground software",
"KAIST and Google AI introduce Blockwise Parallel Decoding",
"Machine Learning Latency Optimization",
"Meta's AI training controversy with Ray-Ban",
"Nvidia and Accenture in AI Innovation",
"Nvidia launches ChatGPT rival NVLM",
"Nvidia spatial AI",
"Nvidia-Accenture AI partnership",
"OpenAI $6.6 billion funding",
"OpenAI F

------------
------------


In [23]:
test_state["cluster_topics"] = [
"AI Regulation in California",
"AI and Military Robotics",
"AI assistants leaking workplace secrets",
"AI in Software Development Funding",
"AI startup drama at Y Combinator",
"AI-Generated Podcasts and Content Creation",
"AI-powered malware evolves with image recognition",
"Apparate ML optimization",
"Cathie Wood's Ark invests in OpenAI",
"Character.AI shifts focus after Google deal",
"Crescendo raises $50M for AI customer service",
"Google's Gemini AI on Android",
"Intel updates AI Playground software",
"KAIST and Google AI introduce Blockwise Parallel Decoding",
"Machine Learning Latency Optimization",
"Meta's AI training controversy with Ray-Ban",
"Nvidia",
"OpenAI Funding and Valuation",
"Poolside $500M Series B for AI coding tools",
"US Army tests AI-powered robot dogs",
"Waymo launches robotaxi rides in Austin"
]


In [24]:
FINAL_SUMMARY_PROMPT = """You are ASA, an advanced summarization assistant designed to
write a compelling summary of news input. You are able to categorize information,
and identify important themes from large volumes of news.

ASA Objective:

I will provide today's news items about AI and summary bullet points in markdown format,
structured according to an input format template.

News items are delimited by ~~~

You are tasked with using the news items to create a concise summary of today's most important topics and developments.

You will write an engaging summary of today's news encompassing the most important and frequently
mentioned topics and themes, in an output format provided below.

ASA Input Item Format Template:

[Story-Title-s1 - source-name-s1](url-s1)

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

Example ASA Input Item Format:

[Apple Intelligence is now live in public beta. Heres what it offers and how to enable it. - TechCrunch](https://techcrunch.com/2024/09/19/apple-intelligence-is-now-live-in-public-beta-heres-what-it-offers-and-how-to-enable-it)

Topics: Apple, Big Tech, Features, Gen AI, Intelligence, Machine Learning, Products, Public Beta, Virtual Assistants

- Apple Intelligence is now live in public beta for users in the U.S. enrolled in the public beta program, featuring generative AI capabilities like advanced writing tools and a revamped Siri.
- The platform is currently only available in U.S. English and is not accessible in the EU or China due to regulatory issues; it supports iPhone 15 Pro, Pro Max, and the new iPhone 16 line.
- Key features include photo editing tools like "Clean Up," a Smart Reply function in Mail, and improvements to Siri’s understanding and on-device task knowledge.

ASA Output Format Template:

# Engaging-topic-title-1

- item-title-1a - [source-name-1a](item-url-1a)
- item-title-1b - [source-name-1b](item-url-1b)
- item-title-1c - [source-name-1c](item-url-1c)

# Engaging-topic-title-2

- item-title-2a - [source-name-2a](item-url-2a)
- item-title-2b - [source-name-2b](item-url-2b)

Example ASA Output Format:

# A military AI revolution

- Eric Schmidt on AI warfare - [FT](https://www.ft.com/content/fe136479-9504-4588-869f-900f2b3452c4)
- Killer robots are real in Ukraine war. - [Yahoo News](https://uk.news.yahoo.com/ai-killer-robots-warning-ukraine-war-133415411.html)

ASA Instructions:
Read the input closely.
USE ONLY INFORMATION PROVIDED IN THE INPUT.
Group news items into related topics.
Each topic should have a snappy, punchy, clever, alliterative, possibly punny title.
Each output item bullet should contain one sentence with one link.
Each topic chould contain the most significant facts from the news items without commentary or elaboration.
Each output item bullet should not repeat points or information from previous bullet points.
You will write each item in the professional but engaging, narrative style of a tech reporter
for a national publication, providing balanced, professional, informative, providing accurate,
clear, concise summaries in a neutral tone.

Check carefully that you only use information provided in the input below, that you include
a link in each output item, and that any bullet point does not repeat information or links previously provided.

Topic suggestions:
{cat_str}

Input:
{bullet_str}

"""

In [25]:
def fn_compose_summary(state: AgentState) -> AgentState:

    cat_str = "\n".join(state['cluster_topics'])
    bullet_str = "\n~~~\n".join(state["bullets"])

    model = ChatOpenAI(
        model=HIGHCOST_MODEL,
        temperature=0.3,
        model_kwargs={"response_format": {"type": "json_object"}}
    )

    chain = ChatPromptTemplate.from_template(FINAL_SUMMARY_PROMPT) | model | SimpleJsonOutputParser()
    response = chain.invoke({ "cat_str": cat_str, "bullet_str": bullet_str})
    print(response)
    state["summary"] = response.content
    return state


if DEBUG:
    _ = fn_compose_summary(test_state)


2024-10-02 18:48:41,752 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"


BadRequestError: Error code: 400 - {'error': {'message': "Unsupported value: 'temperature' does not support 0.3 with this model. Only the default (1) value is supported.", 'type': 'invalid_request_error', 'param': 'temperature', 'code': 'unsupported_value'}}

In [26]:
#post manually to o1-preview and grab response b/c I don't have API yet
cat_str = "\n".join(test_state['cluster_topics'])
bullet_str = "\n~~~\n".join(test_state["bullets"])
final_summary_prompt = FINAL_SUMMARY_PROMPT.format(cat_str=cat_str, bullet_str=bullet_str)
print(final_summary_prompt)


You are ASA, an advanced summarization assistant designed to
write a compelling summary of news input. You are able to categorize information,
and identify important themes from large volumes of news.

ASA Objective:

I will provide today's news items about AI and summary bullet points in markdown format,
structured according to an input format template.

News items are delimited by ~~~

You are tasked with using the news items to create a concise summary of today's most important topics and developments.

You will write an engaging summary of today's news encompassing the most important and frequently
mentioned topics and themes, in an output format provided below.

ASA Input Item Format Template:

[Story-Title-s1 - source-name-s1](url-s1)

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

Example ASA Input Item Format:

[Apple Intelligence is now live in public beta. Heres what it offers and how to enable it. - TechCrunch](https://t

In [27]:
count_tokens(final_summary_prompt)


23861

In [28]:
test_state["summary"] = """
# Power Pair: Accenture and Nvidia Advance Agentic AI

- Nvidia expands partnership with Accenture to enhance AI adoption, establishing an Nvidia Business Group to implement generative AI solutions - [Google News](https://news.google.com/read/CBMisgFBVV95cUxOblU4WlFibVJ5TEVtMHNid2tDQkVJaWd5ZFdzSDl6VDdETlBHaEJDUDRjdm5kcGZTRDFWWUQzUVd1ZllMV0hkOG5Id0JIMGx1MnotYl9rUUpzMmtzUXJEcUg3WE16U1pFN0lyaDA1cklYWUZuNXhRekJzS0xaQ3FoTnBZRmgyTlE1RkZsT3Zna0ZpSy1DdldPdTE2eHcxOWxsWGRDYWRSSXh3d1JUMTVfUDBn)

- Accenture and NVIDIA launch Business Group to train 30,000 professionals globally on AI technologies - [Analytics India Magazine](https://analyticsindiamag.com/ai-news-updates/accenture-and-nvidia-partner-to-train-30000-professionals-to-scale-agentic-ai-for-enterprises/)

- Accenture to train 30,000 employees on Nvidia AI tech and promote enterprise adoption of agentic AI systems - [CRN](https://www.crn.com/news/ai/2024/accenture-to-train-30-000-staff-on-nvidia-ai-tech-in-blockbuster-deal)

# Character Shift: AI's New Chapter Post-Google Deal

- Character.ai stops developing AI models after $2.7 billion deal with Google, focusing on chatbots instead - [Financial Times](https://www.ft.com/content/f2a9b5d4-05fe-4134-b4fe-c24727b85bba)

- Character.ai pivots away from large language models due to high training costs post-Google acquisition - [Google News](https://news.google.com/read/CBMi3gFBVV95cUxOSzAxeHAtV2Q3Vl9Ec1FTb1VNMHVUQmtYdHdUdG5ETWt2dENobEhXTEN0QnpWUEE2OHppdzlnZXQyVnBwTzVWbGFiZFphTU5oZ0ZkOS1aUWZjMENmd1ItcHRjc2FqSXM0SlF6NmZjS3VzTHdCeTFvZkpPb0ZULWtYNU9nZENldl9mZXU1S2gzMW9hdXpaTXdDcGNHUWY5TFF2UDcycnBxcGZQSzZxbFdZajlxQW9NY0FObmZjNkRUcjhMYWhMQ2ZiTUdHMFZsbHBmRDJKem8wZGotaEdOVXc)

# OpenAI's Funding Frenzy: $6.6 Billion Boost

- OpenAI raises $6.6 billion, nearly doubling valuation to $157 billion, with investments from Microsoft, SoftBank, and Nvidia - [The Wall Street Journal](https://www.wsj.com/tech/ai/openai-nearly-doubles-valuation-to-157-billion-in-funding-round-ee220607)

- Cathie Wood's Ark invests at least $250 million in OpenAI's new funding round, increasing its stake to about 5% of total assets - [Business Insider](https://www.businessinsider.com/cathie-wood-ark-venture-openai-funding-2024-10)

- Tiger Global backs OpenAI in new $6.6 billion investment round, participating with prior investors - [Business Insider](https://www.businessinsider.com/tiger-global-openai-latest-funding-round-2024-10)

# Code Cash: Poolside Dives into $500M Funding

- AI coding startup Poolside raises $500 million in Series B led by Bain Capital Ventures at a $3B valuation - [Bloomberg](https://www.bloomberg.com/news/articles/2024-10-02/poolside-raises-500-million-with-bain-dst-for-coding-ai)

- Poolside plans to build AI-assisted software development tools, deploying 10,000 Nvidia GPUs for model training - [TechCrunch](https://techcrunch.com/2024/10/02/ai-coding-startup-poolside-raises-500m-from-ebay-nvidia-and-others)

# Army's Automated Arsenal: Robot Dogs Get AI Guns

- US Army tests AI-powered robot dogs equipped with rifles at Red Sands Integrated Experimentation Center in Saudi Arabia - [Futurism](https://futurism.com/the-byte/army-robot-dogs-guns-middle-east)

- The Army is testing robot dogs with AI guns in the Middle East for anti-drone capabilities - [Google News](https://news.google.com/read/CBMiowFBVV95cUxQeHpodUZQakRlSWwzbWVHdmNvUGVmTTdpSXpYMFVKVTloX2M3UWtKV3dwaVM3NDZqSFpsb0lYM2RQZ0lHUUxObEVxa3R2MmN1RDRaR2dpdFNiV2ZXU0RkUXFXdk5aMFRmQThSRUlsbl8yb0F5NGVjNGctREpFTGw3c1JQRkhVQ0NwOU9KSlBoUlJSOFNBem5iRzlRZWdPSm9BUkFR)

# Apparate Accelerates: ML Latency Leaps

- Apparate introduces early-exit models for ML latency and throughput optimization, achieving significant latency reductions without compromising accuracy - [Hacker Noon](https://hackernoon.com/apparate-early-exit-models-for-ml-latency-and-throughput-optimization-overall-results)

- Apparate demonstrates up to 94.2% latency reduction in CV workloads, optimizing ML inference with adaptive early exits - [Hacker Noon](https://hackernoon.com/apparate-early-exit-models-for-ml-latency-and-throughput-optimization-evaluation-and-methodology)

# AI Podcasts: From Text to Talk with Google's NotebookLM

- Google's NotebookLM can generate convincing AI podcasts from uploaded documents, showcasing advanced language processing capabilities - [Wired](https://www.wired.com/story/ai-podcast-google-notebooklm/)

- Users create AI-generated podcasts using Google's NotebookLM, raising both fascination and concerns over AI's mimicry of human conversation - [Business Insider](https://www.businessinsider.com/googles-ai-podcast-generated-incredibly-convincing-creepy-mimic-humans-talking-2024-10)

# California's AI Bill Blocked: Regulation on Hold

- California Governor Gavin Newsom vetoes landmark AI regulation bill SB 1047, calling for further assessment of AI risks - [Fast Company](https://www.fastcompany.com/91201228/california-spiked-a-landmark-ai-regulation-but-that-doesnt-mean-the-bill-is-going-away)

- Newsom's veto signals a shift in the AI safety debate, focusing on practical issues over existential risks - [Politico](https://www.politico.com/newsletters/digital-future-daily/2024/10/02/what-gavin-newsom-just-did-to-the-global-ai-debate-00182196)
"""

display(Markdown(test_state["summary"].replace("$","\\\\$")))



# Power Pair: Accenture and Nvidia Advance Agentic AI

- Nvidia expands partnership with Accenture to enhance AI adoption, establishing an Nvidia Business Group to implement generative AI solutions - [Google News](https://news.google.com/read/CBMisgFBVV95cUxOblU4WlFibVJ5TEVtMHNid2tDQkVJaWd5ZFdzSDl6VDdETlBHaEJDUDRjdm5kcGZTRDFWWUQzUVd1ZllMV0hkOG5Id0JIMGx1MnotYl9rUUpzMmtzUXJEcUg3WE16U1pFN0lyaDA1cklYWUZuNXhRekJzS0xaQ3FoTnBZRmgyTlE1RkZsT3Zna0ZpSy1DdldPdTE2eHcxOWxsWGRDYWRSSXh3d1JUMTVfUDBn)

- Accenture and NVIDIA launch Business Group to train 30,000 professionals globally on AI technologies - [Analytics India Magazine](https://analyticsindiamag.com/ai-news-updates/accenture-and-nvidia-partner-to-train-30000-professionals-to-scale-agentic-ai-for-enterprises/)

- Accenture to train 30,000 employees on Nvidia AI tech and promote enterprise adoption of agentic AI systems - [CRN](https://www.crn.com/news/ai/2024/accenture-to-train-30-000-staff-on-nvidia-ai-tech-in-blockbuster-deal)

# Character Shift: AI's New Chapter Post-Google Deal

- Character.ai stops developing AI models after \\$2.7 billion deal with Google, focusing on chatbots instead - [Financial Times](https://www.ft.com/content/f2a9b5d4-05fe-4134-b4fe-c24727b85bba)

- Character.ai pivots away from large language models due to high training costs post-Google acquisition - [Google News](https://news.google.com/read/CBMi3gFBVV95cUxOSzAxeHAtV2Q3Vl9Ec1FTb1VNMHVUQmtYdHdUdG5ETWt2dENobEhXTEN0QnpWUEE2OHppdzlnZXQyVnBwTzVWbGFiZFphTU5oZ0ZkOS1aUWZjMENmd1ItcHRjc2FqSXM0SlF6NmZjS3VzTHdCeTFvZkpPb0ZULWtYNU9nZENldl9mZXU1S2gzMW9hdXpaTXdDcGNHUWY5TFF2UDcycnBxcGZQSzZxbFdZajlxQW9NY0FObmZjNkRUcjhMYWhMQ2ZiTUdHMFZsbHBmRDJKem8wZGotaEdOVXc)

# OpenAI's Funding Frenzy: \\$6.6 Billion Boost

- OpenAI raises \\$6.6 billion, nearly doubling valuation to \\$157 billion, with investments from Microsoft, SoftBank, and Nvidia - [The Wall Street Journal](https://www.wsj.com/tech/ai/openai-nearly-doubles-valuation-to-157-billion-in-funding-round-ee220607)

- Cathie Wood's Ark invests at least \\$250 million in OpenAI's new funding round, increasing its stake to about 5% of total assets - [Business Insider](https://www.businessinsider.com/cathie-wood-ark-venture-openai-funding-2024-10)

- Tiger Global backs OpenAI in new \\$6.6 billion investment round, participating with prior investors - [Business Insider](https://www.businessinsider.com/tiger-global-openai-latest-funding-round-2024-10)

# Code Cash: Poolside Dives into \\$500M Funding

- AI coding startup Poolside raises \\$500 million in Series B led by Bain Capital Ventures at a \\$3B valuation - [Bloomberg](https://www.bloomberg.com/news/articles/2024-10-02/poolside-raises-500-million-with-bain-dst-for-coding-ai)

- Poolside plans to build AI-assisted software development tools, deploying 10,000 Nvidia GPUs for model training - [TechCrunch](https://techcrunch.com/2024/10/02/ai-coding-startup-poolside-raises-500m-from-ebay-nvidia-and-others)

# Army's Automated Arsenal: Robot Dogs Get AI Guns

- US Army tests AI-powered robot dogs equipped with rifles at Red Sands Integrated Experimentation Center in Saudi Arabia - [Futurism](https://futurism.com/the-byte/army-robot-dogs-guns-middle-east)

- The Army is testing robot dogs with AI guns in the Middle East for anti-drone capabilities - [Google News](https://news.google.com/read/CBMiowFBVV95cUxQeHpodUZQakRlSWwzbWVHdmNvUGVmTTdpSXpYMFVKVTloX2M3UWtKV3dwaVM3NDZqSFpsb0lYM2RQZ0lHUUxObEVxa3R2MmN1RDRaR2dpdFNiV2ZXU0RkUXFXdk5aMFRmQThSRUlsbl8yb0F5NGVjNGctREpFTGw3c1JQRkhVQ0NwOU9KSlBoUlJSOFNBem5iRzlRZWdPSm9BUkFR)

# Apparate Accelerates: ML Latency Leaps

- Apparate introduces early-exit models for ML latency and throughput optimization, achieving significant latency reductions without compromising accuracy - [Hacker Noon](https://hackernoon.com/apparate-early-exit-models-for-ml-latency-and-throughput-optimization-overall-results)

- Apparate demonstrates up to 94.2% latency reduction in CV workloads, optimizing ML inference with adaptive early exits - [Hacker Noon](https://hackernoon.com/apparate-early-exit-models-for-ml-latency-and-throughput-optimization-evaluation-and-methodology)

# AI Podcasts: From Text to Talk with Google's NotebookLM

- Google's NotebookLM can generate convincing AI podcasts from uploaded documents, showcasing advanced language processing capabilities - [Wired](https://www.wired.com/story/ai-podcast-google-notebooklm/)

- Users create AI-generated podcasts using Google's NotebookLM, raising both fascination and concerns over AI's mimicry of human conversation - [Business Insider](https://www.businessinsider.com/googles-ai-podcast-generated-incredibly-convincing-creepy-mimic-humans-talking-2024-10)

# California's AI Bill Blocked: Regulation on Hold

- California Governor Gavin Newsom vetoes landmark AI regulation bill SB 1047, calling for further assessment of AI risks - [Fast Company](https://www.fastcompany.com/91201228/california-spiked-a-landmark-ai-regulation-but-that-doesnt-mean-the-bill-is-going-away)

- Newsom's veto signals a shift in the AI safety debate, focusing on practical issues over existential risks - [Politico](https://www.politico.com/newsletters/digital-future-daily/2024/10/02/what-gavin-newsom-just-did-to-the-global-ai-debate-00182196)


In [ ]:
REWRITE_PROMPT = """You will act as a professional editor with a strong background in technology journalism.
You have a deep understanding of current and emerging AI trends, and the ability to
produce, edit, and curate high-quality content that engages and informs readers. You are
especially skilled at reviewing and enhancing tech writing, helping improve clarity, conciseness,
and coherence, and ensuring its accuracy and relevance.

Objective: The markdown newsletter provided below contains several sections consisting of bullet points.
Carefully review each section of the newsletter. Edit the newsletter for issues according
to the detailed instructions below, and respond with the updated newsletter or 'OK' if no changes
are needed.

Instructions:
For each section, remove or combine bullet points which are highly duplicative or redundant.
Make bullet points as concise as possible, sticking to facts without editorial comment.
For each section, review the title. The title should be snappy, punchy, clever, possibly alliterative or punny.
The section title most be short, engaging, clever, and as consistent with the bullets in the section as possible
At the top of the newsletter add an overall title synthesizing several top news items.
Respond with the updated newsletter only in markdown format, or the word 'OK' if no changes are needed.

Newsletter to edit:
{summary}

"""

In [ ]:
def fn_rewrite_summary(state: AgentState) -> AgentState:

    model = ChatOpenAI(
        model=HIGHCOST_MODEL,
        temperature=0.3,
        model_kwargs={"response_format": {"type": "json_object"}}
    )

    chain = ChatPromptTemplate.from_template(REWRITE_PROMPT) | model | SimpleJsonOutputParser()
    response = chain.invoke({ "summary": state["summary"]})
    print(response)
    state["summary"] = response.content
    return state


if DEBUG:
    _ = fn_rewrite_summary(test_state)


In [ ]:
rewrite_prompt = (REWRITE_PROMPT.format(summary=test_state["summary"]))
print(rewrite_prompt)


In [ ]:
test_state["summmary"] = """
# AI Ascends: Funding Frenzy, Apple's Awakening, Copilot's Comeback

## Funding Frenzy Fuels AI Automation

- **11x.ai** raises $50M to expand AI sales bots - [TechCrunch](https://techcrunch.com/2024/09/30/11x-ai-a-developer-of-ai-sales-reps-has-raised-50m-series-b-led-by-a16z-sources-say/)
- **Numa** secures $32M to automate car dealerships - [TechCrunch](https://techcrunch.com/2024/10/01/numa-is-bringing-ai-and-automation-to-car-dealerships/)
- **Resolve AI** lands $35M to revolutionize software troubleshooting - [Reuters](https://www.reuters.com/technology/artificial-intelligence/greylock-backed-resolve-ai-raises-35-million-seed-funding-help-engineers-2024-10-01/)
- **Crescendo** raises $50M and acquires PartnerHero for AI-driven contact centers - [Bloomberg](https://www.bloomberg.com/news/articles/2024-10-02/ai-startup-hits-500-million-valuation-to-rival-contact-centers)
- **Dexory** grabs $80M to boost autonomous warehouse robots - [Sifted](https://sifted.eu/articles/ai-robots-dexory-80m-raise-news/)
- **Ameba** raises $7M to streamline retail supply chains with AI - [TechCrunch](https://techcrunch.com/2024/10/02/hedosophia-leads-7m-seed-round-into-retail-supply-chain-ai-startup-ameba/)

## Apple's AI Awakening

- **Apple** unveils upcoming AI features for iPhones, with Apple Intelligence launching on iPhone SE early next year - [The New York Times](https://www.nytimes.com/2024/10/02/technology/personaltech/apple-intelligence-iphone.html), [Google News](https://news.google.com/articles/CBMiiAFBVV95cUxQZnFvSTRCXzVUMXprbWpTaTN0UFJoeE9sNHZVdEREcmxvMVdOaE16dDhfRlIyYjZTYjdsQ0tlOWNUSkJ5THVmMWhSOEN3bEh3TDdxZ3h5NFh0ODNBLU1ENVVHWnVMRjFiUU9idGNKZFZxbVRlYUR6MEVxWEp2RkcxdjZxTmZ3ZDJG)

## Copilot's Clever Comeback

- **Microsoft** unveils major updates to Copilot and Windows AI features, including voice and vision capabilities - [The Verge](https://www.theverge.com/2024/10/1/24259261/microsoft-event-copilot-ai-windows), [VentureBeat](https://venturebeat.com/ai/microsoft-leans-harder-into-ai-updating-copilot-bing-and-windows/), [TechCrunch](https://techcrunch.com/2024/10/01/microsoft-copilot-can-now-read-your-screen-think-deeper-and-speak-aloud-to-you/), [Google News](https://news.google.com/articles/CBMilwFBVV95cUxObkZLMkpydVFCdzVRdExJWmxDSDhhbUxTZ0JESE9RdGhCQk1hZVk3bG1IeDVmMzMtREVEOFdqSzdrSlNPWDdnSkFjZGJvcXg0cXpTdUcxcFhUTDdsSlZOVVVvV0hIdUZmUlRDeFV1UXJjMGVZTVJQV2Y0RDhiOHVzMm11RE40Z2VnNUZOcDRYWmZCVHlGdzY0)
- **Windows 11** update adds AI to Paint and Photos - [Engadget](https://www.engadget.com/computing/microsoft-delivers-new-copilot-ai-pc-features-with-windows-11s-2024-update-140048807.html)
- **Microsoft** introduces Click to Do and AI-enhanced search - [The Verge](https://www.theverge.com/2024/10/1/24258337/microsoft-windows-ai-features-click-to-do-super-resolution-photos)
- **Microsoft** pays publishers for content in Copilot Daily - [TechCrunch](https://techcrunch.com/2024/10/01/microsoft-starts-paying-publishers-for-content-in-copilot/)
- **Copilot Labs vs. Google Labs**: A comparison - [ZDNet](https://www.zdnet.com/article/what-is-microsofts-copilot-labs-and-how-does-it-compare-to-google-labs/#ftag=RSSbaffb68)

## Military's Mechanical Mutts: Robot Dogs Deployed

- **U.S. Army** tests armed robot dogs with AI-enabled weapons in the Middle East - [Reddit](https://www.reddit.com/r/technology/comments/1ftrfvv/the_army_has_sent_an_armed_robot_dog_to_the/), [Google News](https://news.google.com/articles/CBMiowFBVV95cUxQM2UyRG1HSDJMQ1JkaUhkODhtZWxTRlFtS24xS3dIYVNTdUlveE1PTTluN2VYa19pMXVpSnBTYXpLNGlteVpkNTh2aDJZcnROaDQxMUg0cHJEU2l5NlgwTWNnaTFMbThsVWtxRGZGUEVkaFI4TE9SdllhQnRYVjU2SFp1UTVkZmtvNXByVk94ajBMYzZVVTdvQXFUZW84QkMxeGI00gGoAUFVX3lxTE5BS1NFeTctUnI5QTBacXpjU0tZbkFfQVhqVXV2YXh0bWx2WTgxSTNucTRtbVR3VnV4YU5QNU1BLVllUXFadFQwQUtxY0tuVzRWNzB1M0tjelUyYTZjVFRYT3ZzOExYVnJoM0VXVUxHLVg5ekxrM0ZVejJwbmVaeHZCQ0xaOFNnb3hzSFl1eTdqbFBYbjR4UnVuakkwRENFMThhYk82bnE4WQ)

## California's AI Conundrum

- **Governor Newsom** vetoes California AI Safety Bill, sparking uncertainty - [The Hill](https://thehill.com/policy/technology/4910138-newsom-throws-ai-regulation-fight-into-uncertainty-with-veto/), [Schneier](https://www.schneier.com/blog/archives/2024/10/california-ai-safety-bill-vetoed.html)
- **California** passes law protecting consumer brain data - [Reddit](https://www.reddit.com/r/technology/comments/1ftxw0h/california_passes_law_protecting_consumer_brain/)

## Machine Learning Magic: Optimization Tricks Unveiled

- **Apparate** proposes early-exit models for ML optimization - [Hacker Noon](https://hackernoon.com/apparate-early-exit-models-for-ml-latency-and-throughput-optimization-abstract-and-introduction)
- **Archon** promises quicker LLMs without extra costs - [VentureBeat](https://venturebeat.com/ai/inference-framework-archon-promises-to-make-llms-quicker-without-additional-costs/)
"""


if DEBUG:
    display(Markdown(test_state["summary"].replace("$","\\\\$")))


In [ ]:
def fn_is_revision_complete(state: AgentState) -> str:
    """update edit_complete if MAX_EDITS exceeded"
    return "complete" if edit_complete else "incomplete"
    """

    if state["n_edits"] >= MAX_EDITS:
        log("Max edits reached")
        state["edit_complete"] = True


    return "complete" if state["edit_complete"] else "incomplete"


if DEBUG:
    
    MAX_EDITS=3
    test_state["n_edits"]=0
    test_state["edit_complete"]=False
    
    print(fn_is_revision_complete(test_state))


In [ ]:
def fn_send_mail(state: AgentState) -> AgentState:

    log("Sending summary email")
    # Convert Markdown to HTML
    html_str = markdown.markdown(state['summary'], extensions=['extra'])
    # send email
    subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
    send_gmail(subject, html_str)
    return state

if DEBUG:
    fn_send_mail(test_state)

In [ ]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        self.graph = graph
        graph.add_node("initialize", self.initialize)
        graph.add_node("download_sources", self.download_sources)
        graph.add_node("extract_urls", self.extract_urls)
        graph.add_node("filter_urls", self.filter_urls)
        graph.add_node("topic_analysis", self.topic_analysis)
        graph.add_node("topic_clusters", self.topic_clusters)
        graph.add_node("download_pages", self.download_pages)
        graph.add_node("summarize_pages", self.summarize_pages)
        graph.add_node("propose_topics", self.propose_topics)
        graph.add_node("compose_summary", self.compose_summary)
        graph.add_node("rewrite_summary", self.rewrite_summary)
        graph.add_node("send_mail", self.send_mail)

        graph.add_edge(START, "initialize")
        graph.add_edge("initialize", "download_sources")
        graph.add_edge("download_sources", "extract_urls")
        graph.add_edge("extract_urls", "filter_urls")
        graph.add_edge("filter_urls", "topic_analysis")
        graph.add_edge("topic_analysis", "topic_clusters")
        graph.add_edge("topic_clusters", "download_pages")
        graph.add_edge("download_pages", "summarize_pages")
        graph.add_edge("summarize_pages", "propose_topics")
        graph.add_edge("propose_topics", "compose_summary")
        graph.add_edge("compose_summary", "rewrite_summary")
        graph.add_conditional_edges("rewrite_summary",
                                    self.is_revision_complete,
                                    {"incomplete": "rewrite_summary",
                                     "complete": "send_mail",
                                    })
        graph.add_edge("send_mail", END)

        checkpointer = SqliteSaver.from_conn_string(":memory:")
        app = graph.compile(checkpointer=checkpointer, interrupt_before=["filter_urls", "compose_summary",])
        self.app = app

#         self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def initialize(self, state: AgentState) -> AgentState:
        self.state = fn_initialize(state)
        return self.state

    def download_sources(self, state: AgentState) -> AgentState:
        self.state = fn_download_sources(state)
        return self.state

    def extract_urls(self, state: AgentState) -> AgentState:
        self.state = fn_extract_urls(state)
        return self.state

    def filter_urls(self, state: AgentState) -> AgentState:
        self.state = fn_filter_urls(state)
        return self.state

    def topic_analysis(self, state: AgentState) -> AgentState:
        self.state = fn_topic_analysis(state)
        return self.state

    def topic_clusters(self, state: AgentState) -> AgentState:
        self.state = fn_topic_clusters(state)
        return self.state

    def download_pages(self, state: AgentState) -> AgentState:
        self.state = fn_download_pages(state)
        return self.state

    def summarize_pages(self, state: AgentState) -> AgentState:
        self.state = fn_summarize_pages(state)
        return self.state

    def propose_topics(self, state: AgentState) -> AgentState:
        self.state = fn_propose_cats(state)
        return self.state

    def compose_summary(self, state: AgentState) -> AgentState:
        self.state = fn_compose_summary(state)
        return self.state

    def rewrite_summary(self, state: AgentState) -> AgentState:
        self.state = fn_rewrite_summary(state)
        return self.state

    def is_revision_complete(self, state: AgentState) -> str:
        return fn_is_revision_complete(state)

    def send_mail(self, state: AgentState) -> AgentState:
        self.state = fn_send_mail(state)
        return self.state

    def run(self, config, state):
        self.state = state
        for step in self.app.stream(state, config, stream_mode="debug"):
            if step["type"] == "checkpoint":
                display(f'Step {step["step"]}')

            # Check if there is an error message in the state
#             if step["payload"].get("values") and step["payload"]["values"].get("error_message"):
#                 # Update the error state to 'tested' since error simulation is complete
#                 self.state['error_state'] = 'tested'
#                 # Retry from the last checkpoint saved by the checkpointer
#                 for retry_step in graph.stream(self.state, config, stream_mode="debug"):
#                     if retry_step["type"] == "checkpoint":
#                         print(retry_step["step"], retry_step["payload"].get("values"))
#                 break
        return None


In [ ]:
model = ChatOpenAI(model=MODEL)
lg_agent = Agent(model, [], system="")


In [ ]:
# Image(lg_agent.graph.get_graph().draw_png())
Image(lg_agent.app.get_graph().draw_mermaid_png())


In [ ]:
do_download = True
before_date = None
# before_date="2024-07-22 10:00:00"


In [ ]:
# Configuration with thread ID for checkpointing
# Generate a random UUID
thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}
# initial state
state = AgentState()
state["before_date"] = before_date
state["do_download"] = do_download
log(f"Starting with before_date={state.get('before_date')}, do_download={do_download}, thread_id={thread_id}")
lg_agent.run(config, state)


In [ ]:
user_approval = input("Edit topics? (yes/no): ")


In [ ]:
if user_approval.lower() == "yes":
    # If approved, continue the graph execution
    for step in lg_agent.graph.stream(None, config, stream_mode="debug"):
        if step["type"] == "checkpoint":
            display(f'Step {step["step"]}')
else:
    print("Operation cancelled by user.")


In [ ]:
# get last state
last_state = lg_agent.graph.get_state(config)
pd.DataFrame(last_state.values['AIdf'])


In [ ]:
# update state as node
# lg_agent.graph.update_state(config, {"foo": 2, "bar": ["b"]}, as_node="revise_summary")


In [ ]:
# resume from error starting at a node
# result = lg_agent.graph.invoke(None, {'configurable': {'thread_id': 'newsbot-thread-5'}, "next": "topic_analysis"})